<a href="https://www.kaggle.com/code/vtrackstar/ml-nba-offensive-playstyle-through-11-22-24?scriptVersionId=209380037" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Ask
* 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans


pd.set_option('display.max_columns', None)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/updated/2024 NBA Scoring (11_22) - PerGame (1).csv
/kaggle/input/nbaplayerstat112324/2024 NBA Scoring (11_22) - Shooting.csv
/kaggle/input/nbaplayerstat112324/2024 NBA Scoring (11_22) - PerGame.csv
/kaggle/input/nbaplayerstat112324/2024 NBA Scoring (11_22) - Play by Play.csv


## Prepare

In [2]:
perg = pd.read_csv('/kaggle/input/updated/2024 NBA Scoring (11_22) - PerGame (1).csv')
shooting = pd.read_csv('/kaggle/input/nbaplayerstat112324/2024 NBA Scoring (11_22) - Play by Play.csv')
playbplay = pd.read_csv('/kaggle/input/nbaplayerstat112324/2024 NBA Scoring (11_22) - Shooting.csv')

In [3]:
perg

,Rk,Player,Age,Team,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Awards
0,1,Giannis Antetokounmpo,30,MIL,PF,15,15,35.3,13.1,21.4,0.614,0.1,0.7,0.182,13.0,20.7,0.629,0.617,6.0,10.1,0.596,2.1,9.9,12.0,6.4,0.6,1.4,3.2,3.0,32.4,NaN
1,2,Anthony Davis,31,LAL,C,14,14,35.2,10.9,19.1,0.573,0.9,2.1,0.433,10.0,16.9,0.591,0.597,8.5,10.9,0.783,2.6,8.6,11.2,2.8,1.2,1.9,2.1,1.9,31.3,NaN
2,3,Nikola Jokić,29,DEN,C,11,11,38.3,11.0,19.5,0.565,2.2,3.7,0.585,8.8,15.7,0.561,0.621,5.8,7.0,0.831,4.6,9.4,14.0,11.5,1.5,0.9,4.0,1.8,30.0,NaN
3,4,Jayson Tatum,26,BOS,PF,16,16,36.6,9.1,20.1,0.455,4.1,10.9,0.371,5.1,9.1,0.555,0.556,6.8,8.4,0.806,0.4,7.8,8.2,6.1,1.4,0.5,2.9,2.6,29.1,NaN
4,5,Paolo Banchero,22,ORL,PF,5,5,36.4,9.6,19.4,0.495,2.2,6.4,0.344,7.4,13.0,0.569,0.552,7.6,11.8,0.644,2.4,6.4,8.8,5.6,0.6,0.8,2.2,2.6,29.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,296,Jamal Cain,25,NOP,SF,12,0,15.3,1.6,3.7,0.432,0.8,2.0,0.375,0.8,1.7,0.500,0.534,0.8,1.2,0.643,0.9,2.0,2.9,0.8,0.4,0.4,0.6,1.5,4.7,NaN
296,297,Drew Eubanks,27,UTA,C,12,0,14.9,1.9,3.0,0.639,0.1,0.3,0.333,1.8,2.8,0.667,0.653,0.8,1.3,0.600,1.4,2.8,4.3,1.2,0.3,0.7,0.8,1.3,4.7,NaN
297,298,Taylor Hendricks,21,UTA,PF,3,3,25.0,1.3,6.0,0.222,1.0,4.0,0.250,0.3,2.0,0.167,0.306,1.0,1.3,0.750,1.7,3.3,5.0,0.7,1.7,1.3,0.7,2.7,4.7,NaN
298,299,Reggie Jackson,34,PHI,PG,7,0,13.6,1.7,4.7,0.364,1.3,3.1,0.409,0.4,1.6,0.273,0.500,0.0,0.0,NaN,0.0,1.0,1.0,2.0,0.6,0.1,0.7,1.0,4.7,NaN


In [4]:
shooting

,Rk,Player,Age,Team,Pos,G,GS,MP,PG%,SG%,SF%,PF%,C%,OnCourt,On-Off,BadPassTO,LostBallTO,ShootFoulCmtd,OffFoulCmtd,ShootFoulDrawn,OffFoulDrawn,PtsGenbyAsts,And1s,FGA_Blkd,Awards
0,1,Cade Cunningham,23,DET,PG,17,17,618,92.0,8.0,0.0,0.0,0.0,-1.7,-1.2,49,22,18,3,37,2,365,9,24,NaN
1,2,De'Aaron Fox,27,SAC,PG,16,16,604,91.0,9.0,0.0,0.0,0.0,2.9,2.5,21,29,13,4,55,10,228,8,17,NaN
2,3,James Harden,35,LAC,PG,17,17,593,60.0,40.0,0.0,0.0,0.0,-0.3,-6.1,37,27,13,9,46,3,344,6,18,NaN
3,4,Keegan Murray,24,SAC,PF,16,16,590,0.0,0.0,16.0,83.0,0.0,3.7,5.5,9,6,26,1,11,4,70,2,5,NaN
4,5,Devin Booker,28,PHO,SG,16,16,588,1.0,37.0,56.0,7.0,0.0,-4.6,-8.4,11,13,11,4,39,6,263,5,18,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,396,Karlo Matković,23,NOP,PF,4,0,30,0.0,0.0,0.0,27.0,73.0,-10.4,2.1,3,0,3,1,0,0,0,0,1,NaN
396,397,Dejounte Murray,28,NOP,PG,1,1,30,18.0,83.0,0.0,0.0,0.0,6.1,19.2,0,1,0,0,3,0,22,0,0,NaN
397,398,MarJon Beauchamp,24,MIL,SF,10,0,29,0.0,0.0,65.0,35.0,0.0,20.6,21.9,2,1,2,1,2,0,2,0,1,NaN
398,399,Tyler Smith,20,MIL,PF,9,0,29,0.0,0.0,0.0,60.0,40.0,19.9,21.2,0,0,2,1,1,1,0,0,0,NaN


In [5]:
playbplay

,Rk,Player,Age,Team,Pos,G,GS,MP,FG%,AvgFGDist,%FGA_2P,%FGA_0-3,%FGA_3-10,%FGA_10-16,%FGA_16-3P,%FGA_3P,FG%_2P,FG%_0-3,FG%_3-10,FG%_10-16,FG%_16-3P,FG%_3P,%FG_Astd_2P,%FG_Astd_3P,%FGA_Dunks,Num_Dunks,%3PA_Heaves,3P%_Heaves,Num_Heaves_Att,Num_Heaves_Made,Awards
0,1,Cade Cunningham,23,DET,PG,17,17,618,0.438,12.8,0.688,0.212,0.291,0.129,0.056,0.312,0.474,0.528,0.455,0.455,0.421,0.358,0.243,0.605,0.021,3,0.075,0.500,1,0,NaN
1,2,De'Aaron Fox,27,SAC,PG,16,16,604,0.506,13.8,0.699,0.133,0.304,0.166,0.096,0.301,0.578,0.773,0.564,0.509,0.469,0.340,0.246,0.500,0.015,5,0.150,0.267,1,0,NaN
2,3,James Harden,35,LAC,PG,17,17,593,0.365,16.8,0.502,0.151,0.196,0.144,0.011,0.498,0.419,0.561,0.321,0.385,0.667,0.311,0.175,0.524,0.007,2,0.037,0.400,0,0,NaN
3,4,Keegan Murray,24,SAC,PF,16,16,590,0.426,16.6,0.497,0.137,0.191,0.109,0.060,0.503,0.549,0.920,0.400,0.400,0.455,0.304,0.640,1.000,0.082,14,0.337,0.387,0,0,NaN
4,5,Devin Booker,28,PHO,SG,16,16,588,0.438,16.6,0.580,0.101,0.177,0.160,0.142,0.420,0.509,0.690,0.471,0.457,0.488,0.339,0.435,0.707,0.010,2,0.116,0.286,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,396,Karlo Matković,23,NOP,PF,4,0,30,0.667,2.2,1.000,0.667,0.333,0.000,0.000,0.000,0.667,1.000,0.000,NaN,NaN,NaN,0.500,NaN,0.000,0,NaN,NaN,0,0,NaN
396,397,Dejounte Murray,28,NOP,PG,1,1,30,0.267,11.0,0.800,0.267,0.333,0.133,0.067,0.200,0.333,0.750,0.000,0.000,1.000,0.000,0.250,NaN,0.000,0,0.000,NaN,0,0,NaN
397,398,MarJon Beauchamp,24,MIL,SF,10,0,29,0.333,15.4,0.533,0.267,0.133,0.133,0.000,0.467,0.500,0.750,0.000,0.500,NaN,0.143,0.250,1.000,0.133,1,0.000,NaN,0,0,NaN
398,399,Tyler Smith,20,MIL,PF,9,0,29,0.667,16.2,0.417,0.167,0.250,0.000,0.000,0.583,1.000,1.000,1.000,NaN,NaN,0.429,0.800,0.667,0.167,2,0.286,0.500,0,0,NaN


In [6]:
print(perg.columns)
print(shooting.columns)
print(playbplay.columns)

Index(['Rk', 'Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       'Awards'],
      dtype='object')
Index(['Rk', 'Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'PG%', 'SG%',
       'SF%', 'PF%', 'C%', 'OnCourt', 'On-Off', 'BadPassTO', 'LostBallTO',
       'ShootFoulCmtd', 'OffFoulCmtd', 'ShootFoulDrawn', 'OffFoulDrawn',
       'PtsGenbyAsts', 'And1s', 'FGA_Blkd', 'Awards'],
      dtype='object')
Index(['Rk', 'Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'FG%',
       'AvgFGDist', '%FGA_2P', '%FGA_0-3', '%FGA_3-10', '%FGA_10-16',
       '%FGA_16-3P', '%FGA_3P', 'FG%_2P', 'FG%_0-3', 'FG%_3-10', 'FG%_10-16',
       'FG%_16-3P', 'FG%_3P', '%FG_Astd_2P', '%FG_Astd_3P', '%FGA_Dunks',
       'Num_Dunks', '%3PA_Heaves', '3P%_Heaves', 'Num_Heaves_Att',
       'Num_Heaves_Made', 'Awards'],
      dtype='object')


In [7]:
nullperg = perg.isnull().sum()
nullsht = shooting.isnull().sum()
nullpbp = playbplay.isnull().sum()
print(nullperg)
print(nullsht)
print(nullpbp)

Rk          0
Player      0
Age         0
Team        0
Pos         0
G           0
GS          0
MP          0
FG          0
FGA         0
FG%         0
3P          0
3PA         0
3P%        17
2P          0
2PA         0
2P%         0
eFG%        0
FT          0
FTA         0
FT%         3
ORB         0
DRB         0
TRB         0
AST         0
STL         0
BLK         0
TOV         0
PF          0
PTS         0
Awards    300
dtype: int64
Rk                  0
Player              0
Age                 0
Team                0
Pos                 0
G                   0
GS                  0
MP                  0
PG%                 1
SG%                 1
SF%                 1
PF%                 1
C%                  1
OnCourt             0
On-Off              0
BadPassTO           0
LostBallTO          0
ShootFoulCmtd       0
OffFoulCmtd         0
ShootFoulDrawn      0
OffFoulDrawn        0
PtsGenbyAsts        0
And1s               0
FGA_Blkd            0
Awards            400
dty

In [8]:
pergclean = perg.fillna(0)
shtclean = shooting.fillna(0)
pbpclean = playbplay.fillna(0)

In [9]:
nullperg1 = pergclean.isnull().sum()
nullsht1 = shtclean.isnull().sum()
nullpbp1 = pbpclean.isnull().sum()
print(nullperg1)
print(nullsht1)
print(nullpbp1)

Rk        0
Player    0
Age       0
Team      0
Pos       0
G         0
GS        0
MP        0
FG        0
FGA       0
FG%       0
3P        0
3PA       0
3P%       0
2P        0
2PA       0
2P%       0
eFG%      0
FT        0
FTA       0
FT%       0
ORB       0
DRB       0
TRB       0
AST       0
STL       0
BLK       0
TOV       0
PF        0
PTS       0
Awards    0
dtype: int64
Rk                0
Player            0
Age               0
Team              0
Pos               0
G                 0
GS                0
MP                0
PG%               0
SG%               0
SF%               0
PF%               0
C%                0
OnCourt           0
On-Off            0
BadPassTO         0
LostBallTO        0
ShootFoulCmtd     0
OffFoulCmtd       0
ShootFoulDrawn    0
OffFoulDrawn      0
PtsGenbyAsts      0
And1s             0
FGA_Blkd          0
Awards            0
dtype: int64
Rk                 0
Player             0
Age                0
Team               0
Pos               

In [10]:
pergame = pergclean[['Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'MP', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
       'FT%', 'ORB', 'AST', 'TOV', 'PTS']]
shot = shtclean[['Player', 'PG%', 'SG%',
       'SF%', 'PF%', 'C%', 'BadPassTO', 'LostBallTO',
       'OffFoulCmtd', 'ShootFoulDrawn',
       'PtsGenbyAsts', 'And1s', 'FGA_Blkd']]
playbyplay = pbpclean[['Player', 'AvgFGDist', '%FGA_2P', '%FGA_0-3', '%FGA_3-10', '%FGA_10-16',
       '%FGA_16-3P', '%FGA_3P', 'FG%_2P', 'FG%_0-3', 'FG%_3-10', 'FG%_10-16',
       'FG%_16-3P', 'FG%_3P', '%FG_Astd_2P', '%FG_Astd_3P', '%FGA_Dunks',
       'Num_Dunks']]
merged = pd.merge(pergame, shot, on='Player', how='left').merge(playbyplay, on='Player', how='left')

In [11]:
merged

,Player,Age,Team,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,AST,TOV,PTS,PG%,SG%,SF%,PF%,C%,BadPassTO,LostBallTO,OffFoulCmtd,ShootFoulDrawn,PtsGenbyAsts,And1s,FGA_Blkd,AvgFGDist,%FGA_2P,%FGA_0-3,%FGA_3-10,%FGA_10-16,%FGA_16-3P,%FGA_3P,FG%_2P,FG%_0-3,FG%_3-10,FG%_10-16,FG%_16-3P,FG%_3P,%FG_Astd_2P,%FG_Astd_3P,%FGA_Dunks,Num_Dunks
0,Giannis Antetokounmpo,30,MIL,PF,15,15,35.3,13.1,21.4,0.614,0.1,0.7,0.182,13.0,20.7,0.629,0.617,6.0,10.1,0.596,2.1,6.4,3.2,32.4,0.0,0.0,0.0,63.0,37.0,15.0,16.0,11.0,84.0,254.0,27.0,19.0,6.5,0.966,0.570,0.159,0.072,0.165,0.034,0.629,0.765,0.373,0.478,0.472,0.182,0.379,0.000,0.190,59.0
1,Anthony Davis,31,LAL,C,14,14,35.2,10.9,19.1,0.573,0.9,2.1,0.433,10.0,16.9,0.591,0.597,8.5,10.9,0.783,2.6,2.8,2.1,31.3,0.0,0.0,0.0,6.0,94.0,14.0,10.0,2.0,60.0,95.0,15.0,14.0,8.2,0.888,0.378,0.326,0.101,0.082,0.112,0.591,0.782,0.506,0.333,0.364,0.433,0.614,1.000,0.127,31.0
2,Nikola Jokić,29,DEN,C,11,11,38.3,11.0,19.5,0.565,2.2,3.7,0.585,8.8,15.7,0.561,0.621,5.8,7.0,0.831,4.6,11.5,4.0,30.0,0.0,0.0,0.0,0.0,100.0,22.0,17.0,2.0,28.0,303.0,3.0,14.0,9.2,0.808,0.280,0.453,0.065,0.009,0.192,0.561,0.617,0.546,0.357,1.000,0.585,0.577,0.875,0.014,2.0
3,Jayson Tatum,26,BOS,PF,16,16,36.6,9.1,20.1,0.455,4.1,10.9,0.371,5.1,9.1,0.555,0.556,6.8,8.4,0.806,0.4,6.1,2.9,29.1,0.0,2.0,48.0,51.0,0.0,23.0,15.0,6.0,45.0,256.0,8.0,14.0,17.9,0.455,0.150,0.150,0.078,0.078,0.545,0.555,0.875,0.417,0.320,0.440,0.371,0.333,0.492,0.050,15.0
4,Paolo Banchero,22,ORL,PF,5,5,36.4,9.6,19.4,0.495,2.2,6.4,0.344,7.4,13.0,0.569,0.552,7.6,11.8,0.644,2.4,5.6,2.2,29.0,0.0,0.0,0.0,98.0,2.0,6.0,4.0,0.0,29.0,70.0,5.0,6.0,12.9,0.670,0.340,0.134,0.113,0.082,0.330,0.569,0.697,0.385,0.364,0.625,0.344,0.486,0.545,0.041,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,Jamal Cain,25,NOP,SF,12,0,15.3,1.6,3.7,0.432,0.8,2.0,0.375,0.8,1.7,0.500,0.534,0.8,1.2,0.643,0.9,0.8,0.6,4.7,0.0,30.0,52.0,17.0,0.0,5.0,0.0,1.0,7.0,22.0,2.0,0.0,15.7,0.455,0.205,0.182,0.045,0.023,0.545,0.500,0.778,0.375,0.000,0.000,0.375,0.800,1.000,0.000,0.0
296,Drew Eubanks,27,UTA,C,12,0,14.9,1.9,3.0,0.639,0.1,0.3,0.333,1.8,2.8,0.667,0.653,0.8,1.3,0.600,1.4,1.2,0.8,4.7,0.0,0.0,0.0,10.0,90.0,3.0,4.0,1.0,7.0,35.0,0.0,1.0,7.1,0.917,0.333,0.472,0.111,0.000,0.083,0.667,0.750,0.588,0.750,0.000,0.333,0.818,0.000,0.194,6.0
297,Taylor Hendricks,21,UTA,PF,3,3,25.0,1.3,6.0,0.222,1.0,4.0,0.250,0.3,2.0,0.167,0.306,1.0,1.3,0.750,1.7,0.7,0.7,4.7,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,4.0,0.0,0.0,18.8,0.333,0.056,0.167,0.111,0.000,0.667,0.167,0.000,0.333,0.000,0.000,0.250,1.000,1.000,0.056,1.0
298,Reggie Jackson,34,PHI,PG,7,0,13.6,1.7,4.7,0.364,1.3,3.1,0.409,0.4,1.6,0.273,0.500,0.0,0.0,0.000,0.0,2.0,0.7,4.7,91.0,9.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,34.0,0.0,2.0,20.2,0.333,0.121,0.121,0.061,0.030,0.667,0.273,0.250,0.250,0.500,0.000,0.409,0.000,0.778,0.000,0.0


In [12]:
duplicate_count = merged.duplicated().sum()

# Print the result
print(f"Number of duplicate rows: {duplicate_count}")

Number of duplicate rows: 0


In [13]:
conditions = [
    (merged['C%'] > 50) | ((merged['PF%'] > 65) & (merged['SF%'] < 5)),
    (merged['PG%'] > 50) | ((merged['SG%'] > 1) & (merged['SF%'] < 6)),
]

choices = ['Bigs', 'Guards']
merged['Position Group'] = np.select(conditions, choices, default='Wings')

In [14]:
bigs1 = merged[merged['Position Group'] == 'Bigs']
guards1 = merged[merged['Position Group'] == 'Guards']
wings1 = merged[merged['Position Group'] == 'Wings']
bigs = bigs1.copy()
guards = guards1.copy()
wings = wings1.copy()

In [15]:
bigs['%FGA_0-10'] = bigs['%FGA_0-3'] + bigs['%FGA_3-10']
bigs

,Player,Age,Team,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,AST,TOV,PTS,PG%,SG%,SF%,PF%,C%,BadPassTO,LostBallTO,OffFoulCmtd,ShootFoulDrawn,PtsGenbyAsts,And1s,FGA_Blkd,AvgFGDist,%FGA_2P,%FGA_0-3,%FGA_3-10,%FGA_10-16,%FGA_16-3P,%FGA_3P,FG%_2P,FG%_0-3,FG%_3-10,FG%_10-16,FG%_16-3P,FG%_3P,%FG_Astd_2P,%FG_Astd_3P,%FGA_Dunks,Num_Dunks,Position Group,%FGA_0-10
1,Anthony Davis,31,LAL,C,14,14,35.2,10.9,19.1,0.573,0.9,2.1,0.433,10.0,16.9,0.591,0.597,8.5,10.9,0.783,2.6,2.8,2.1,31.3,0.0,0.0,0.0,6.0,94.0,14.0,10.0,2.0,60.0,95.0,15.0,14.0,8.2,0.888,0.378,0.326,0.101,0.082,0.112,0.591,0.782,0.506,0.333,0.364,0.433,0.614,1.000,0.127,31.0,Bigs,0.704
2,Nikola Jokić,29,DEN,C,11,11,38.3,11.0,19.5,0.565,2.2,3.7,0.585,8.8,15.7,0.561,0.621,5.8,7.0,0.831,4.6,11.5,4.0,30.0,0.0,0.0,0.0,0.0,100.0,22.0,17.0,2.0,28.0,303.0,3.0,14.0,9.2,0.808,0.280,0.453,0.065,0.009,0.192,0.561,0.617,0.546,0.357,1.000,0.585,0.577,0.875,0.014,2.0,Bigs,0.733
4,Paolo Banchero,22,ORL,PF,5,5,36.4,9.6,19.4,0.495,2.2,6.4,0.344,7.4,13.0,0.569,0.552,7.6,11.8,0.644,2.4,5.6,2.2,29.0,0.0,0.0,0.0,98.0,2.0,6.0,4.0,0.0,29.0,70.0,5.0,6.0,12.9,0.670,0.340,0.134,0.113,0.082,0.330,0.569,0.697,0.385,0.364,0.625,0.344,0.486,0.545,0.041,4.0,Bigs,0.474
10,Kevin Durant,36,PHO,PF,9,9,38.8,9.9,17.9,0.553,2.7,6.2,0.429,7.2,11.7,0.619,0.627,5.1,6.1,0.836,0.2,3.4,3.3,27.6,0.0,0.0,0.0,84.0,16.0,17.0,6.0,1.0,26.0,76.0,9.0,3.0,15.0,0.652,0.155,0.130,0.292,0.075,0.348,0.619,0.920,0.667,0.468,0.500,0.429,0.492,0.750,0.037,6.0,Bigs,0.285
11,Karl-Anthony Towns,29,NYK,C,14,14,33.0,9.6,17.5,0.551,2.6,5.3,0.500,7.0,12.2,0.573,0.627,4.9,5.7,0.850,2.8,2.9,1.9,26.8,0.0,0.0,0.0,0.0,100.0,8.0,7.0,8.0,43.0,93.0,14.0,18.0,11.2,0.698,0.343,0.278,0.061,0.016,0.302,0.573,0.631,0.559,0.467,0.000,0.500,0.541,0.973,0.073,17.0,Bigs,0.621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,Kevon Looney,28,GSW,C,13,0,15.5,2.2,4.4,0.491,0.0,0.0,0.000,2.2,4.4,0.491,0.491,1.1,2.0,0.538,3.9,1.7,0.6,5.4,0.0,0.0,0.0,0.0,100.0,2.0,3.0,3.0,9.0,55.0,2.0,2.0,4.0,1.000,0.596,0.263,0.053,0.088,0.000,0.491,0.618,0.400,0.000,0.200,0.000,0.429,0.000,0.088,5.0,Bigs,0.859
286,Torrey Craig,34,CHI,SF,4,1,11.3,1.8,4.0,0.438,1.0,3.3,0.308,0.8,0.8,1.000,0.563,0.5,0.5,1.000,0.5,0.8,0.0,5.0,0.0,0.0,3.0,89.0,8.0,0.0,0.0,0.0,1.0,6.0,0.0,0.0,21.8,0.188,0.125,0.063,0.000,0.000,0.813,1.000,1.000,1.000,0.000,0.000,0.308,1.000,0.750,0.000,0.0,Bigs,0.188
291,Mason Plumlee,34,PHO,C,15,3,17.6,1.9,3.3,0.580,0.0,0.0,0.000,1.9,3.3,0.580,0.580,1.1,2.0,0.533,1.6,1.8,0.9,4.9,0.0,0.0,0.0,4.0,96.0,1.0,4.0,6.0,10.0,65.0,2.0,11.0,3.0,1.000,0.580,0.380,0.040,0.000,0.000,0.580,0.828,0.263,0.000,0.000,0.000,0.724,0.000,0.260,12.0,Bigs,0.960
296,Drew Eubanks,27,UTA,C,12,0,14.9,1.9,3.0,0.639,0.1,0.3,0.333,1.8,2.8,0.667,0.653,0.8,1.3,0.600,1.4,1.2,0.8,4.7,0.0,0.0,0.0,10.0,90.0,3.0,4.0,1.0,7.0,35.0,0.0,1.0,7.1,0.917,0.333,0.472,0.111,0.000,0.083,0.667,0.750,0.588,0.750,0.000,0.333,0.818,0.000,0.194,6.0,Bigs,0.805


In [16]:
guards

,Player,Age,Team,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,AST,TOV,PTS,PG%,SG%,SF%,PF%,C%,BadPassTO,LostBallTO,OffFoulCmtd,ShootFoulDrawn,PtsGenbyAsts,And1s,FGA_Blkd,AvgFGDist,%FGA_2P,%FGA_0-3,%FGA_3-10,%FGA_10-16,%FGA_16-3P,%FGA_3P,FG%_2P,FG%_0-3,FG%_3-10,FG%_10-16,FG%_16-3P,FG%_3P,%FG_Astd_2P,%FG_Astd_3P,%FGA_Dunks,Num_Dunks,Position Group
5,LaMelo Ball,23,CHO,PG,15,15,33.1,9.9,23.1,0.428,4.5,12.5,0.356,5.4,10.5,0.513,0.525,4.7,5.3,0.886,1.0,6.7,4.2,28.9,73.0,26.0,1.0,0.0,0.0,35.0,11.0,8.0,33.0,253.0,7.0,15.0,17.6,0.457,0.173,0.182,0.081,0.020,0.543,0.513,0.617,0.492,0.429,0.143,0.356,0.235,0.567,0.009,2.0,Guards
6,De'Aaron Fox,27,SAC,PG,16,16,37.8,10.5,20.8,0.506,2.1,6.3,0.340,8.4,14.5,0.578,0.557,5.7,6.8,0.835,1.0,5.8,3.6,28.8,91.0,9.0,0.0,0.0,0.0,21.0,29.0,4.0,55.0,228.0,8.0,17.0,13.8,0.699,0.133,0.304,0.166,0.096,0.301,0.578,0.773,0.564,0.509,0.469,0.340,0.246,0.500,0.015,5.0,Guards
8,Luka Dončić,25,DAL,PG,14,14,36.6,9.9,22.6,0.435,3.2,9.9,0.324,6.6,12.7,0.522,0.506,5.1,6.6,0.783,0.6,7.6,3.3,28.1,79.0,21.0,0.0,0.0,0.0,32.0,12.0,1.0,45.0,251.0,10.0,7.0,16.5,0.562,0.104,0.230,0.167,0.060,0.438,0.522,0.788,0.534,0.415,0.316,0.324,0.323,0.489,0.000,0.0,Guards
13,Jalen Brunson,28,NYK,PG,15,15,34.3,8.7,18.3,0.476,2.5,6.3,0.404,6.2,12.1,0.514,0.545,5.2,6.3,0.830,0.5,7.3,2.3,25.2,100.0,0.0,0.0,0.0,0.0,23.0,9.0,3.0,38.0,257.0,13.0,14.0,14.3,0.658,0.175,0.225,0.167,0.091,0.342,0.514,0.667,0.468,0.413,0.520,0.404,0.237,0.447,0.000,0.0,Guards
14,Tyrese Maxey,24,PHI,PG,9,9,36.0,9.0,21.7,0.415,2.8,9.9,0.281,6.2,11.8,0.528,0.479,4.4,5.3,0.833,0.1,3.8,1.7,25.2,64.0,36.0,0.0,0.0,0.0,6.0,3.0,2.0,18.0,88.0,7.0,10.0,16.0,0.544,0.179,0.221,0.077,0.067,0.456,0.528,0.571,0.512,0.667,0.308,0.281,0.196,0.320,0.026,5.0,Guards
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,Alex Caruso,30,OKC,SG,13,0,19.7,2.0,5.6,0.356,0.6,3.0,0.205,1.4,2.6,0.529,0.411,0.4,0.5,0.714,0.7,2.2,0.7,5.0,67.0,34.0,0.0,0.0,0.0,6.0,0.0,2.0,3.0,70.0,1.0,4.0,14.8,0.466,0.301,0.137,0.027,0.000,0.534,0.529,0.591,0.400,0.500,0.000,0.205,0.611,0.875,0.027,2.0,Guards
287,Tre Jones,25,SAS,PG,4,0,17.5,1.3,4.0,0.313,0.5,1.5,0.333,0.8,2.5,0.300,0.375,2.0,3.0,0.667,0.3,2.5,1.0,5.0,93.0,7.0,0.0,0.0,0.0,3.0,1.0,0.0,2.0,26.0,1.0,1.0,12.4,0.625,0.250,0.188,0.188,0.000,0.375,0.300,0.750,0.000,0.000,0.000,0.333,0.667,1.000,0.000,0.0,Guards
293,Jared Butler,24,WAS,SG,10,0,8.2,1.7,3.4,0.500,0.1,0.5,0.200,1.6,2.9,0.552,0.515,1.3,1.9,0.684,0.2,2.4,0.3,4.8,70.0,31.0,0.0,0.0,0.0,2.0,0.0,1.0,8.0,52.0,4.0,8.0,8.6,0.853,0.382,0.382,0.059,0.029,0.147,0.552,0.615,0.462,1.000,0.000,0.200,0.188,1.000,0.000,0.0,Guards
294,Lonzo Ball,27,CHI,PG,3,0,15.7,1.7,4.7,0.357,1.3,4.0,0.333,0.3,0.7,0.500,0.500,0.0,0.0,0.000,0.0,3.7,1.0,4.7,100.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,29.0,0.0,0.0,23.0,0.143,0.071,0.071,0.000,0.000,0.857,0.500,1.000,0.000,0.000,0.000,0.333,1.000,1.000,0.000,0.0,Guards


In [17]:
wings['%FGA_Astd'] = wings['%FG_Astd_2P'] + wings['%FG_Astd_3P']
wings.fillna(0, inplace=True)
wings

,Player,Age,Team,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,AST,TOV,PTS,PG%,SG%,SF%,PF%,C%,BadPassTO,LostBallTO,OffFoulCmtd,ShootFoulDrawn,PtsGenbyAsts,And1s,FGA_Blkd,AvgFGDist,%FGA_2P,%FGA_0-3,%FGA_3-10,%FGA_10-16,%FGA_16-3P,%FGA_3P,FG%_2P,FG%_0-3,FG%_3-10,FG%_10-16,FG%_16-3P,FG%_3P,%FG_Astd_2P,%FG_Astd_3P,%FGA_Dunks,Num_Dunks,Position Group,%FGA_Astd
0,Giannis Antetokounmpo,30,MIL,PF,15,15,35.3,13.1,21.4,0.614,0.1,0.7,0.182,13.0,20.7,0.629,0.617,6.0,10.1,0.596,2.1,6.4,3.2,32.4,0.0,0.0,0.0,63.0,37.0,15.0,16.0,11.0,84.0,254.0,27.0,19.0,6.5,0.966,0.570,0.159,0.072,0.165,0.034,0.629,0.765,0.373,0.478,0.472,0.182,0.379,0.000,0.190,59.0,Wings,0.379
3,Jayson Tatum,26,BOS,PF,16,16,36.6,9.1,20.1,0.455,4.1,10.9,0.371,5.1,9.1,0.555,0.556,6.8,8.4,0.806,0.4,6.1,2.9,29.1,0.0,2.0,48.0,51.0,0.0,23.0,15.0,6.0,45.0,256.0,8.0,14.0,17.9,0.455,0.150,0.150,0.078,0.078,0.545,0.555,0.875,0.417,0.320,0.440,0.371,0.333,0.492,0.050,15.0,Wings,0.825
7,Shai Gilgeous-Alexander,26,OKC,SG,16,16,34.1,9.9,19.9,0.500,1.8,5.4,0.337,8.1,14.5,0.560,0.546,7.0,7.8,0.896,0.9,6.3,2.6,28.7,0.0,11.0,59.0,28.0,2.0,25.0,10.0,0.0,52.0,254.0,10.0,12.0,12.1,0.730,0.258,0.252,0.167,0.053,0.270,0.560,0.768,0.400,0.528,0.412,0.337,0.208,0.207,0.022,7.0,Wings,0.415
9,Anthony Edwards,23,MIN,SG,15,15,37.1,9.9,20.9,0.475,4.8,11.1,0.431,5.1,9.8,0.524,0.589,3.3,4.3,0.766,0.5,3.7,3.1,27.9,2.0,69.0,28.0,0.0,0.0,20.0,16.0,3.0,35.0,133.0,11.0,10.0,17.6,0.468,0.159,0.156,0.061,0.092,0.532,0.524,0.640,0.531,0.316,0.448,0.431,0.312,0.500,0.029,8.0,Wings,0.812
12,Jaylen Brown,28,BOS,SF,12,12,36.9,8.7,20.3,0.426,2.1,7.3,0.284,6.6,13.0,0.506,0.477,5.9,7.7,0.772,1.6,4.4,2.5,25.3,0.0,16.0,83.0,1.0,0.0,7.0,14.0,9.0,40.0,134.0,4.0,13.0,14.4,0.639,0.164,0.262,0.131,0.082,0.361,0.506,0.675,0.453,0.438,0.450,0.284,0.316,0.680,0.037,8.0,Wings,0.996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,Pat Connaughton,32,MIL,SG,13,0,17.6,1.7,4.8,0.355,0.9,3.1,0.300,0.8,1.7,0.455,0.452,0.6,0.8,0.800,0.5,1.8,0.8,4.9,2.0,37.0,60.0,2.0,0.0,9.0,1.0,0.0,3.0,56.0,0.0,5.0,17.7,0.355,0.210,0.129,0.016,0.000,0.645,0.455,0.615,0.250,0.000,0.000,0.300,1.000,0.917,0.016,1.0,Wings,1.917
290,Pelle Larsson,23,MIA,SG,8,0,13.1,1.8,3.3,0.538,0.6,1.5,0.417,1.1,1.8,0.643,0.635,0.8,1.3,0.600,0.3,1.4,0.5,4.9,14.0,39.0,45.0,3.0,0.0,2.0,1.0,0.0,5.0,29.0,2.0,1.0,13.3,0.538,0.269,0.269,0.000,0.000,0.462,0.643,0.714,0.571,0.000,0.000,0.417,0.778,1.000,0.038,1.0,Wings,1.778
292,Dalen Terry,22,CHI,SG,17,0,13.8,1.6,3.2,0.491,0.5,1.1,0.444,1.1,2.2,0.514,0.564,1.2,1.6,0.778,0.8,1.1,1.0,4.9,2.0,21.0,72.0,5.0,0.0,9.0,3.0,0.0,12.0,48.0,1.0,4.0,10.0,0.673,0.527,0.109,0.000,0.036,0.327,0.514,0.552,0.333,0.000,0.500,0.444,0.579,1.000,0.055,3.0,Wings,1.579
295,Jamal Cain,25,NOP,SF,12,0,15.3,1.6,3.7,0.432,0.8,2.0,0.375,0.8,1.7,0.500,0.534,0.8,1.2,0.643,0.9,0.8,0.6,4.7,0.0,30.0,52.0,17.0,0.0,5.0,0.0,1.0,7.0,22.0,2.0,0.0,15.7,0.455,0.205,0.182,0.045,0.023,0.545,0.500,0.778,0.375,0.000,0.000,0.375,0.800,1.000,0.000,0.0,Wings,1.800


In [18]:
print(wings.isnull().sum())

Player            0
Age               0
Team              0
Pos               0
G                 0
GS                0
MP                0
FG                0
FGA               0
FG%               0
3P                0
3PA               0
3P%               0
2P                0
2PA               0
2P%               0
eFG%              0
FT                0
FTA               0
FT%               0
ORB               0
AST               0
TOV               0
PTS               0
PG%               0
SG%               0
SF%               0
PF%               0
C%                0
BadPassTO         0
LostBallTO        0
OffFoulCmtd       0
ShootFoulDrawn    0
PtsGenbyAsts      0
And1s             0
FGA_Blkd          0
AvgFGDist         0
%FGA_2P           0
%FGA_0-3          0
%FGA_3-10         0
%FGA_10-16        0
%FGA_16-3P        0
%FGA_3P           0
FG%_2P            0
FG%_0-3           0
FG%_3-10          0
FG%_10-16         0
FG%_16-3P         0
FG%_3P            0
%FG_Astd_2P       0


## Process

In [19]:
## Feature Engineering
bigs_features = bigs[['ORB','%FGA_3P', 'AvgFGDist', '%FGA_0-10', 'PTS', '%FGA_Dunks']]
guards_features = guards[['PTS', 'AST', 'PtsGenbyAsts', 'FGA', 'FTA']]
wings_features = wings[['PTS', '%FGA_Astd', 'FGA', 'FTA']]

In [20]:
scaler = StandardScaler()

bigs_scaled = pd.DataFrame(scaler.fit_transform(bigs_features), columns=bigs_features.columns)
guards_scaled = pd.DataFrame(scaler.fit_transform(guards_features), columns=guards_features.columns)
wings_scaled = pd.DataFrame(scaler.fit_transform(wings_features), columns=wings_features.columns)

## Analyze and Model

In [21]:
kmeans_bigs = KMeans(n_clusters=3, random_state=42, n_init=15)
bigs['Cluster'] = kmeans_bigs.fit_predict(bigs_scaled)

kmeans_guards = KMeans(n_clusters=3, random_state=42, n_init=15)
guards['Cluster'] = kmeans_guards.fit_predict(guards_scaled)

kmeans_wings = KMeans(n_clusters=3, random_state=42, n_init=15)
wings['Cluster'] = kmeans_wings.fit_predict(wings_scaled)

In [22]:
guards

,Player,Age,Team,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,AST,TOV,PTS,PG%,SG%,SF%,PF%,C%,BadPassTO,LostBallTO,OffFoulCmtd,ShootFoulDrawn,PtsGenbyAsts,And1s,FGA_Blkd,AvgFGDist,%FGA_2P,%FGA_0-3,%FGA_3-10,%FGA_10-16,%FGA_16-3P,%FGA_3P,FG%_2P,FG%_0-3,FG%_3-10,FG%_10-16,FG%_16-3P,FG%_3P,%FG_Astd_2P,%FG_Astd_3P,%FGA_Dunks,Num_Dunks,Position Group,Cluster
5,LaMelo Ball,23,CHO,PG,15,15,33.1,9.9,23.1,0.428,4.5,12.5,0.356,5.4,10.5,0.513,0.525,4.7,5.3,0.886,1.0,6.7,4.2,28.9,73.0,26.0,1.0,0.0,0.0,35.0,11.0,8.0,33.0,253.0,7.0,15.0,17.6,0.457,0.173,0.182,0.081,0.020,0.543,0.513,0.617,0.492,0.429,0.143,0.356,0.235,0.567,0.009,2.0,Guards,1
6,De'Aaron Fox,27,SAC,PG,16,16,37.8,10.5,20.8,0.506,2.1,6.3,0.340,8.4,14.5,0.578,0.557,5.7,6.8,0.835,1.0,5.8,3.6,28.8,91.0,9.0,0.0,0.0,0.0,21.0,29.0,4.0,55.0,228.0,8.0,17.0,13.8,0.699,0.133,0.304,0.166,0.096,0.301,0.578,0.773,0.564,0.509,0.469,0.340,0.246,0.500,0.015,5.0,Guards,1
8,Luka Dončić,25,DAL,PG,14,14,36.6,9.9,22.6,0.435,3.2,9.9,0.324,6.6,12.7,0.522,0.506,5.1,6.6,0.783,0.6,7.6,3.3,28.1,79.0,21.0,0.0,0.0,0.0,32.0,12.0,1.0,45.0,251.0,10.0,7.0,16.5,0.562,0.104,0.230,0.167,0.060,0.438,0.522,0.788,0.534,0.415,0.316,0.324,0.323,0.489,0.000,0.0,Guards,1
13,Jalen Brunson,28,NYK,PG,15,15,34.3,8.7,18.3,0.476,2.5,6.3,0.404,6.2,12.1,0.514,0.545,5.2,6.3,0.830,0.5,7.3,2.3,25.2,100.0,0.0,0.0,0.0,0.0,23.0,9.0,3.0,38.0,257.0,13.0,14.0,14.3,0.658,0.175,0.225,0.167,0.091,0.342,0.514,0.667,0.468,0.413,0.520,0.404,0.237,0.447,0.000,0.0,Guards,1
14,Tyrese Maxey,24,PHI,PG,9,9,36.0,9.0,21.7,0.415,2.8,9.9,0.281,6.2,11.8,0.528,0.479,4.4,5.3,0.833,0.1,3.8,1.7,25.2,64.0,36.0,0.0,0.0,0.0,6.0,3.0,2.0,18.0,88.0,7.0,10.0,16.0,0.544,0.179,0.221,0.077,0.067,0.456,0.528,0.571,0.512,0.667,0.308,0.281,0.196,0.320,0.026,5.0,Guards,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,Alex Caruso,30,OKC,SG,13,0,19.7,2.0,5.6,0.356,0.6,3.0,0.205,1.4,2.6,0.529,0.411,0.4,0.5,0.714,0.7,2.2,0.7,5.0,67.0,34.0,0.0,0.0,0.0,6.0,0.0,2.0,3.0,70.0,1.0,4.0,14.8,0.466,0.301,0.137,0.027,0.000,0.534,0.529,0.591,0.400,0.500,0.000,0.205,0.611,0.875,0.027,2.0,Guards,0
287,Tre Jones,25,SAS,PG,4,0,17.5,1.3,4.0,0.313,0.5,1.5,0.333,0.8,2.5,0.300,0.375,2.0,3.0,0.667,0.3,2.5,1.0,5.0,93.0,7.0,0.0,0.0,0.0,3.0,1.0,0.0,2.0,26.0,1.0,1.0,12.4,0.625,0.250,0.188,0.188,0.000,0.375,0.300,0.750,0.000,0.000,0.000,0.333,0.667,1.000,0.000,0.0,Guards,0
293,Jared Butler,24,WAS,SG,10,0,8.2,1.7,3.4,0.500,0.1,0.5,0.200,1.6,2.9,0.552,0.515,1.3,1.9,0.684,0.2,2.4,0.3,4.8,70.0,31.0,0.0,0.0,0.0,2.0,0.0,1.0,8.0,52.0,4.0,8.0,8.6,0.853,0.382,0.382,0.059,0.029,0.147,0.552,0.615,0.462,1.000,0.000,0.200,0.188,1.000,0.000,0.0,Guards,0
294,Lonzo Ball,27,CHI,PG,3,0,15.7,1.7,4.7,0.357,1.3,4.0,0.333,0.3,0.7,0.500,0.500,0.0,0.0,0.000,0.0,3.7,1.0,4.7,100.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,29.0,0.0,0.0,23.0,0.143,0.071,0.071,0.000,0.000,0.857,0.500,1.000,0.000,0.000,0.000,0.333,1.000,1.000,0.000,0.0,Guards,0


In [23]:
wings

,Player,Age,Team,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,AST,TOV,PTS,PG%,SG%,SF%,PF%,C%,BadPassTO,LostBallTO,OffFoulCmtd,ShootFoulDrawn,PtsGenbyAsts,And1s,FGA_Blkd,AvgFGDist,%FGA_2P,%FGA_0-3,%FGA_3-10,%FGA_10-16,%FGA_16-3P,%FGA_3P,FG%_2P,FG%_0-3,FG%_3-10,FG%_10-16,FG%_16-3P,FG%_3P,%FG_Astd_2P,%FG_Astd_3P,%FGA_Dunks,Num_Dunks,Position Group,%FGA_Astd,Cluster
0,Giannis Antetokounmpo,30,MIL,PF,15,15,35.3,13.1,21.4,0.614,0.1,0.7,0.182,13.0,20.7,0.629,0.617,6.0,10.1,0.596,2.1,6.4,3.2,32.4,0.0,0.0,0.0,63.0,37.0,15.0,16.0,11.0,84.0,254.0,27.0,19.0,6.5,0.966,0.570,0.159,0.072,0.165,0.034,0.629,0.765,0.373,0.478,0.472,0.182,0.379,0.000,0.190,59.0,Wings,0.379,1
3,Jayson Tatum,26,BOS,PF,16,16,36.6,9.1,20.1,0.455,4.1,10.9,0.371,5.1,9.1,0.555,0.556,6.8,8.4,0.806,0.4,6.1,2.9,29.1,0.0,2.0,48.0,51.0,0.0,23.0,15.0,6.0,45.0,256.0,8.0,14.0,17.9,0.455,0.150,0.150,0.078,0.078,0.545,0.555,0.875,0.417,0.320,0.440,0.371,0.333,0.492,0.050,15.0,Wings,0.825,1
7,Shai Gilgeous-Alexander,26,OKC,SG,16,16,34.1,9.9,19.9,0.500,1.8,5.4,0.337,8.1,14.5,0.560,0.546,7.0,7.8,0.896,0.9,6.3,2.6,28.7,0.0,11.0,59.0,28.0,2.0,25.0,10.0,0.0,52.0,254.0,10.0,12.0,12.1,0.730,0.258,0.252,0.167,0.053,0.270,0.560,0.768,0.400,0.528,0.412,0.337,0.208,0.207,0.022,7.0,Wings,0.415,1
9,Anthony Edwards,23,MIN,SG,15,15,37.1,9.9,20.9,0.475,4.8,11.1,0.431,5.1,9.8,0.524,0.589,3.3,4.3,0.766,0.5,3.7,3.1,27.9,2.0,69.0,28.0,0.0,0.0,20.0,16.0,3.0,35.0,133.0,11.0,10.0,17.6,0.468,0.159,0.156,0.061,0.092,0.532,0.524,0.640,0.531,0.316,0.448,0.431,0.312,0.500,0.029,8.0,Wings,0.812,1
12,Jaylen Brown,28,BOS,SF,12,12,36.9,8.7,20.3,0.426,2.1,7.3,0.284,6.6,13.0,0.506,0.477,5.9,7.7,0.772,1.6,4.4,2.5,25.3,0.0,16.0,83.0,1.0,0.0,7.0,14.0,9.0,40.0,134.0,4.0,13.0,14.4,0.639,0.164,0.262,0.131,0.082,0.361,0.506,0.675,0.453,0.438,0.450,0.284,0.316,0.680,0.037,8.0,Wings,0.996,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,Pat Connaughton,32,MIL,SG,13,0,17.6,1.7,4.8,0.355,0.9,3.1,0.300,0.8,1.7,0.455,0.452,0.6,0.8,0.800,0.5,1.8,0.8,4.9,2.0,37.0,60.0,2.0,0.0,9.0,1.0,0.0,3.0,56.0,0.0,5.0,17.7,0.355,0.210,0.129,0.016,0.000,0.645,0.455,0.615,0.250,0.000,0.000,0.300,1.000,0.917,0.016,1.0,Wings,1.917,0
290,Pelle Larsson,23,MIA,SG,8,0,13.1,1.8,3.3,0.538,0.6,1.5,0.417,1.1,1.8,0.643,0.635,0.8,1.3,0.600,0.3,1.4,0.5,4.9,14.0,39.0,45.0,3.0,0.0,2.0,1.0,0.0,5.0,29.0,2.0,1.0,13.3,0.538,0.269,0.269,0.000,0.000,0.462,0.643,0.714,0.571,0.000,0.000,0.417,0.778,1.000,0.038,1.0,Wings,1.778,0
292,Dalen Terry,22,CHI,SG,17,0,13.8,1.6,3.2,0.491,0.5,1.1,0.444,1.1,2.2,0.514,0.564,1.2,1.6,0.778,0.8,1.1,1.0,4.9,2.0,21.0,72.0,5.0,0.0,9.0,3.0,0.0,12.0,48.0,1.0,4.0,10.0,0.673,0.527,0.109,0.000,0.036,0.327,0.514,0.552,0.333,0.000,0.500,0.444,0.579,1.000,0.055,3.0,Wings,1.579,0
295,Jamal Cain,25,NOP,SF,12,0,15.3,1.6,3.7,0.432,0.8,2.0,0.375,0.8,1.7,0.500,0.534,0.8,1.2,0.643,0.9,0.8,0.6,4.7,0.0,30.0,52.0,17.0,0.0,5.0,0.0,1.0,7.0,22.0,2.0,0.0,15.7,0.455,0.205,0.182,0.045,0.023,0.545,0.500,0.778,0.375,0.000,0.000,0.375,0.800,1.000,0.000,0.0,Wings,1.800,0


In [24]:
bigs

,Player,Age,Team,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,AST,TOV,PTS,PG%,SG%,SF%,PF%,C%,BadPassTO,LostBallTO,OffFoulCmtd,ShootFoulDrawn,PtsGenbyAsts,And1s,FGA_Blkd,AvgFGDist,%FGA_2P,%FGA_0-3,%FGA_3-10,%FGA_10-16,%FGA_16-3P,%FGA_3P,FG%_2P,FG%_0-3,FG%_3-10,FG%_10-16,FG%_16-3P,FG%_3P,%FG_Astd_2P,%FG_Astd_3P,%FGA_Dunks,Num_Dunks,Position Group,%FGA_0-10,Cluster
1,Anthony Davis,31,LAL,C,14,14,35.2,10.9,19.1,0.573,0.9,2.1,0.433,10.0,16.9,0.591,0.597,8.5,10.9,0.783,2.6,2.8,2.1,31.3,0.0,0.0,0.0,6.0,94.0,14.0,10.0,2.0,60.0,95.0,15.0,14.0,8.2,0.888,0.378,0.326,0.101,0.082,0.112,0.591,0.782,0.506,0.333,0.364,0.433,0.614,1.000,0.127,31.0,Bigs,0.704,2
2,Nikola Jokić,29,DEN,C,11,11,38.3,11.0,19.5,0.565,2.2,3.7,0.585,8.8,15.7,0.561,0.621,5.8,7.0,0.831,4.6,11.5,4.0,30.0,0.0,0.0,0.0,0.0,100.0,22.0,17.0,2.0,28.0,303.0,3.0,14.0,9.2,0.808,0.280,0.453,0.065,0.009,0.192,0.561,0.617,0.546,0.357,1.000,0.585,0.577,0.875,0.014,2.0,Bigs,0.733,2
4,Paolo Banchero,22,ORL,PF,5,5,36.4,9.6,19.4,0.495,2.2,6.4,0.344,7.4,13.0,0.569,0.552,7.6,11.8,0.644,2.4,5.6,2.2,29.0,0.0,0.0,0.0,98.0,2.0,6.0,4.0,0.0,29.0,70.0,5.0,6.0,12.9,0.670,0.340,0.134,0.113,0.082,0.330,0.569,0.697,0.385,0.364,0.625,0.344,0.486,0.545,0.041,4.0,Bigs,0.474,2
10,Kevin Durant,36,PHO,PF,9,9,38.8,9.9,17.9,0.553,2.7,6.2,0.429,7.2,11.7,0.619,0.627,5.1,6.1,0.836,0.2,3.4,3.3,27.6,0.0,0.0,0.0,84.0,16.0,17.0,6.0,1.0,26.0,76.0,9.0,3.0,15.0,0.652,0.155,0.130,0.292,0.075,0.348,0.619,0.920,0.667,0.468,0.500,0.429,0.492,0.750,0.037,6.0,Bigs,0.285,0
11,Karl-Anthony Towns,29,NYK,C,14,14,33.0,9.6,17.5,0.551,2.6,5.3,0.500,7.0,12.2,0.573,0.627,4.9,5.7,0.850,2.8,2.9,1.9,26.8,0.0,0.0,0.0,0.0,100.0,8.0,7.0,8.0,43.0,93.0,14.0,18.0,11.2,0.698,0.343,0.278,0.061,0.016,0.302,0.573,0.631,0.559,0.467,0.000,0.500,0.541,0.973,0.073,17.0,Bigs,0.621,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,Kevon Looney,28,GSW,C,13,0,15.5,2.2,4.4,0.491,0.0,0.0,0.000,2.2,4.4,0.491,0.491,1.1,2.0,0.538,3.9,1.7,0.6,5.4,0.0,0.0,0.0,0.0,100.0,2.0,3.0,3.0,9.0,55.0,2.0,2.0,4.0,1.000,0.596,0.263,0.053,0.088,0.000,0.491,0.618,0.400,0.000,0.200,0.000,0.429,0.000,0.088,5.0,Bigs,0.859,1
286,Torrey Craig,34,CHI,SF,4,1,11.3,1.8,4.0,0.438,1.0,3.3,0.308,0.8,0.8,1.000,0.563,0.5,0.5,1.000,0.5,0.8,0.0,5.0,0.0,0.0,3.0,89.0,8.0,0.0,0.0,0.0,1.0,6.0,0.0,0.0,21.8,0.188,0.125,0.063,0.000,0.000,0.813,1.000,1.000,1.000,0.000,0.000,0.308,1.000,0.750,0.000,0.0,Bigs,0.188,0
291,Mason Plumlee,34,PHO,C,15,3,17.6,1.9,3.3,0.580,0.0,0.0,0.000,1.9,3.3,0.580,0.580,1.1,2.0,0.533,1.6,1.8,0.9,4.9,0.0,0.0,0.0,4.0,96.0,1.0,4.0,6.0,10.0,65.0,2.0,11.0,3.0,1.000,0.580,0.380,0.040,0.000,0.000,0.580,0.828,0.263,0.000,0.000,0.000,0.724,0.000,0.260,12.0,Bigs,0.960,1
296,Drew Eubanks,27,UTA,C,12,0,14.9,1.9,3.0,0.639,0.1,0.3,0.333,1.8,2.8,0.667,0.653,0.8,1.3,0.600,1.4,1.2,0.8,4.7,0.0,0.0,0.0,10.0,90.0,3.0,4.0,1.0,7.0,35.0,0.0,1.0,7.1,0.917,0.333,0.472,0.111,0.000,0.083,0.667,0.750,0.588,0.750,0.000,0.333,0.818,0.000,0.194,6.0,Bigs,0.805,1


In [25]:
guardscorer = guards[guards['Cluster'] == 1]
guardbal = guards[guards['Cluster'] == 2]
guardpass = guards[guards['Cluster'] == 0]

wingset = wings[wings['Cluster'] == 0]
wingiso = wings[wings['Cluster'] == 1]
wingbal = wings[wings['Cluster'] == 2]

bigb2b = bigs[bigs['Cluster'] == 1]
bigbal = bigs[bigs['Cluster'] == 2]
bigstretch = bigs[bigs['Cluster'] == 0]

In [26]:
Sguards_score = guardscorer.sort_values(by='PTS', ascending=False)
Sguards_bal = guardbal.sort_values(by='PTS', ascending=False)
Sguards_pass = guardpass.sort_values(by='AST', ascending=False)
Swings_set = wingset.sort_values(by='PTS', ascending=False)
Swings_iso = wingiso.sort_values(by='PTS', ascending=False)
Swings_bal = wingbal.sort_values(by='PTS', ascending=False)
Sbigs_b2b = bigb2b.sort_values(by='PTS', ascending=False)
Sbigs_bal = bigbal.sort_values(by='PTS', ascending=False)
Sbigs_stretch = bigstretch.sort_values(by='PTS', ascending=False)

## Share

In [27]:
print("\nScoring Guards:")
Sguards_score


Scoring Guards:


,Player,Age,Team,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,AST,TOV,PTS,PG%,SG%,SF%,PF%,C%,BadPassTO,LostBallTO,OffFoulCmtd,ShootFoulDrawn,PtsGenbyAsts,And1s,FGA_Blkd,AvgFGDist,%FGA_2P,%FGA_0-3,%FGA_3-10,%FGA_10-16,%FGA_16-3P,%FGA_3P,FG%_2P,FG%_0-3,FG%_3-10,FG%_10-16,FG%_16-3P,FG%_3P,%FG_Astd_2P,%FG_Astd_3P,%FGA_Dunks,Num_Dunks,Position Group,Cluster
5,LaMelo Ball,23,CHO,PG,15,15,33.1,9.9,23.1,0.428,4.5,12.5,0.356,5.4,10.5,0.513,0.525,4.7,5.3,0.886,1.0,6.7,4.2,28.9,73.0,26.0,1.0,0.0,0.0,35.0,11.0,8.0,33.0,253.0,7.0,15.0,17.6,0.457,0.173,0.182,0.081,0.020,0.543,0.513,0.617,0.492,0.429,0.143,0.356,0.235,0.567,0.009,2.0,Guards,1
6,De'Aaron Fox,27,SAC,PG,16,16,37.8,10.5,20.8,0.506,2.1,6.3,0.340,8.4,14.5,0.578,0.557,5.7,6.8,0.835,1.0,5.8,3.6,28.8,91.0,9.0,0.0,0.0,0.0,21.0,29.0,4.0,55.0,228.0,8.0,17.0,13.8,0.699,0.133,0.304,0.166,0.096,0.301,0.578,0.773,0.564,0.509,0.469,0.340,0.246,0.500,0.015,5.0,Guards,1
8,Luka Dončić,25,DAL,PG,14,14,36.6,9.9,22.6,0.435,3.2,9.9,0.324,6.6,12.7,0.522,0.506,5.1,6.6,0.783,0.6,7.6,3.3,28.1,79.0,21.0,0.0,0.0,0.0,32.0,12.0,1.0,45.0,251.0,10.0,7.0,16.5,0.562,0.104,0.230,0.167,0.060,0.438,0.522,0.788,0.534,0.415,0.316,0.324,0.323,0.489,0.000,0.0,Guards,1
13,Jalen Brunson,28,NYK,PG,15,15,34.3,8.7,18.3,0.476,2.5,6.3,0.404,6.2,12.1,0.514,0.545,5.2,6.3,0.830,0.5,7.3,2.3,25.2,100.0,0.0,0.0,0.0,0.0,23.0,9.0,3.0,38.0,257.0,13.0,14.0,14.3,0.658,0.175,0.225,0.167,0.091,0.342,0.514,0.667,0.468,0.413,0.520,0.404,0.237,0.447,0.000,0.0,Guards,1
14,Tyrese Maxey,24,PHI,PG,9,9,36.0,9.0,21.7,0.415,2.8,9.9,0.281,6.2,11.8,0.528,0.479,4.4,5.3,0.833,0.1,3.8,1.7,25.2,64.0,36.0,0.0,0.0,0.0,6.0,3.0,2.0,18.0,88.0,7.0,10.0,16.0,0.544,0.179,0.221,0.077,0.067,0.456,0.528,0.571,0.512,0.667,0.308,0.281,0.196,0.320,0.026,5.0,Guards,1
15,Damian Lillard,34,MIL,PG,13,13,35.8,7.7,17.8,0.431,2.8,8.9,0.319,4.8,8.9,0.543,0.511,6.5,7.2,0.914,0.7,7.6,2.8,24.8,100.0,0.0,0.0,0.0,0.0,25.0,10.0,0.0,38.0,234.0,3.0,11.0,17.6,0.500,0.181,0.112,0.086,0.121,0.500,0.543,0.667,0.346,0.600,0.500,0.319,0.143,0.459,0.004,1.0,Guards,1
16,Donovan Mitchell,28,CLE,SG,16,16,31.0,8.7,18.7,0.465,3.6,8.9,0.401,5.1,9.8,0.522,0.560,3.4,4.1,0.831,0.6,3.9,1.9,24.3,41.0,59.0,0.0,0.0,0.0,10.0,14.0,3.0,25.0,156.0,6.0,8.0,17.6,0.525,0.154,0.127,0.094,0.151,0.475,0.522,0.674,0.605,0.321,0.422,0.401,0.220,0.544,0.007,1.0,Guards,1
17,Tyler Herro,25,MIA,SG,13,13,33.8,8.2,16.9,0.486,4.4,9.7,0.452,3.8,7.2,0.532,0.616,3.4,3.8,0.880,0.3,5.2,2.6,24.2,21.0,78.0,1.0,0.0,0.0,15.0,12.0,4.0,23.0,168.0,5.0,13.0,17.5,0.427,0.141,0.214,0.059,0.014,0.573,0.532,0.677,0.447,0.462,0.667,0.452,0.340,0.719,0.014,3.0,Guards,1
23,Cade Cunningham,23,DET,PG,17,17,36.4,8.8,20.0,0.438,2.2,6.2,0.358,6.5,13.8,0.474,0.494,3.7,4.6,0.808,1.1,8.9,4.6,23.5,92.0,8.0,0.0,0.0,0.0,49.0,22.0,3.0,37.0,365.0,9.0,24.0,12.8,0.688,0.212,0.291,0.129,0.056,0.312,0.474,0.528,0.455,0.455,0.421,0.358,0.243,0.605,0.021,3.0,Guards,1
28,Stephen Curry,36,GSW,PG,12,12,29.6,7.6,15.5,0.489,4.1,9.1,0.450,3.5,6.4,0.545,0.621,3.4,3.6,0.953,0.3,6.6,3.7,22.7,100.0,0.0,0.0,0.0,0.0,33.0,6.0,3.0,13.0,182.0,5.0,9.0,18.4,0.414,0.140,0.177,0.059,0.038,0.586,0.545,0.615,0.394,0.727,0.714,0.450,0.429,0.633,0.000,0.0,Guards,1


In [28]:
print("\nBalanced Guards:")
Sguards_bal


Balanced Guards:


,Player,Age,Team,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,AST,TOV,PTS,PG%,SG%,SF%,PF%,C%,BadPassTO,LostBallTO,OffFoulCmtd,ShootFoulDrawn,PtsGenbyAsts,And1s,FGA_Blkd,AvgFGDist,%FGA_2P,%FGA_0-3,%FGA_3-10,%FGA_10-16,%FGA_16-3P,%FGA_3P,FG%_2P,FG%_0-3,FG%_3-10,FG%_10-16,FG%_16-3P,FG%_3P,%FG_Astd_2P,%FG_Astd_3P,%FGA_Dunks,Num_Dunks,Position Group,Cluster
44,Coby White,24,CHI,SG,17,17,33.1,6.5,14.5,0.445,3.5,9.1,0.381,3.0,5.4,0.554,0.565,3.1,3.5,0.883,0.2,4.8,2.6,19.5,39.0,57.0,4.0,0.0,0.0,19.0,10.0,5.0,25.0,202.0,5.0,8.0,19.5,0.372,0.162,0.093,0.061,0.057,0.628,0.554,0.675,0.478,0.733,0.143,0.381,0.333,0.831,0.004,0.0,Guards,2
47,Jalen Green,22,HOU,SG,17,17,32.2,6.7,17.2,0.390,2.6,8.3,0.319,4.1,8.9,0.457,0.467,3.2,3.7,0.857,0.6,2.8,2.6,19.2,12.0,88.0,1.0,0.0,0.0,16.0,21.0,4.0,28.0,107.0,5.0,24.0,16.3,0.517,0.199,0.171,0.045,0.103,0.483,0.457,0.707,0.340,0.154,0.300,0.319,0.304,0.511,0.055,13.0,Guards,2
51,CJ McCollum,33,NOP,PG,4,4,34.3,7.3,16.3,0.446,3.3,8.5,0.382,4.0,7.8,0.516,0.546,1.0,1.5,0.667,0.3,4.0,2.0,18.8,90.0,10.0,0.0,0.0,0.0,3.0,4.0,0.0,3.0,39.0,0.0,5.0,17.8,0.477,0.154,0.123,0.108,0.092,0.523,0.516,1.000,0.250,0.429,0.167,0.382,0.375,0.615,0.000,0.0,Guards,2
58,Jamal Murray,27,DEN,PG,11,11,36.5,6.3,15.8,0.397,2.3,6.7,0.338,4.0,9.1,0.440,0.468,3.1,3.6,0.850,0.5,6.0,2.2,17.9,69.0,31.0,0.0,0.0,0.0,18.0,4.0,0.0,14.0,163.0,3.0,11.0,17.4,0.575,0.109,0.161,0.132,0.172,0.425,0.440,0.684,0.429,0.391,0.333,0.338,0.409,0.760,0.011,2.0,Guards,2
68,Dennis Schröder,31,BRK,PG,16,16,34.3,5.9,13.3,0.448,2.6,6.4,0.408,3.3,6.8,0.486,0.547,2.7,3.2,0.843,0.1,6.5,2.4,17.2,100.0,0.0,0.0,0.0,0.0,22.0,9.0,3.0,15.0,261.0,2.0,14.0,16.1,0.514,0.217,0.160,0.085,0.052,0.486,0.486,0.609,0.324,0.500,0.455,0.408,0.208,0.690,0.000,0.0,Guards,2
74,Jared McCain,20,PHI,SG,15,5,23.7,5.9,12.1,0.484,2.5,5.9,0.427,3.3,6.2,0.538,0.588,2.3,2.3,0.971,0.6,2.7,1.3,16.5,11.0,88.0,1.0,0.0,0.0,6.0,8.0,3.0,15.0,95.0,3.0,9.0,15.8,0.511,0.209,0.176,0.099,0.027,0.489,0.538,0.632,0.406,0.444,1.000,0.427,0.280,0.816,0.011,2.0,Guards,2
85,Anfernee Simons,25,POR,SG,13,13,29.1,5.4,14.2,0.378,2.1,6.9,0.300,3.3,7.3,0.453,0.451,2.8,3.0,0.923,0.2,3.9,2.5,15.6,57.0,42.0,1.0,0.0,0.0,21.0,8.0,1.0,17.0,126.0,4.0,15.0,16.6,0.514,0.184,0.184,0.114,0.032,0.486,0.453,0.647,0.382,0.190,0.667,0.300,0.233,0.333,0.022,3.0,Guards,2
88,Jalen Suggs,23,ORL,PG,17,17,30.4,5.2,12.8,0.408,2.2,7.1,0.314,3.0,5.7,0.526,0.495,2.8,3.0,0.922,0.6,3.8,2.8,15.5,97.0,3.0,0.0,0.0,0.0,24.0,12.0,4.0,18.0,148.0,4.0,4.0,19.0,0.445,0.151,0.119,0.119,0.055,0.555,0.526,0.727,0.423,0.385,0.500,0.314,0.196,0.947,0.005,1.0,Guards,2
91,Immanuel Quickley,25,TOR,PG,3,3,22.7,5.3,14.0,0.381,2.0,5.3,0.375,3.3,8.7,0.385,0.452,2.7,3.0,0.889,0.3,4.0,1.3,15.3,85.0,15.0,0.0,0.0,0.0,3.0,1.0,0.0,4.0,28.0,1.0,5.0,14.5,0.619,0.143,0.310,0.119,0.048,0.381,0.385,0.333,0.308,0.800,0.000,0.375,0.900,0.500,0.000,0.0,Guards,2
92,Keyonte George,21,UTA,PG,14,14,31.8,5.0,13.8,0.363,2.6,8.1,0.319,2.4,5.7,0.425,0.456,2.6,3.3,0.804,0.5,5.5,2.8,15.2,86.0,14.0,0.0,0.0,0.0,24.0,7.0,2.0,22.0,184.0,5.0,15.0,18.0,0.415,0.088,0.233,0.067,0.026,0.585,0.425,0.647,0.356,0.308,0.600,0.319,0.441,0.778,0.021,3.0,Guards,2


In [29]:
print("\nPlaymaking Guards:")
Sguards_pass


Playmaking Guards:


,Player,Age,Team,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,AST,TOV,PTS,PG%,SG%,SF%,PF%,C%,BadPassTO,LostBallTO,OffFoulCmtd,ShootFoulDrawn,PtsGenbyAsts,And1s,FGA_Blkd,AvgFGDist,%FGA_2P,%FGA_0-3,%FGA_3-10,%FGA_10-16,%FGA_16-3P,%FGA_3P,FG%_2P,FG%_0-3,FG%_3-10,FG%_10-16,FG%_16-3P,FG%_3P,%FG_Astd_2P,%FG_Astd_3P,%FGA_Dunks,Num_Dunks,Position Group,Cluster
231,Elfrid Payton,30,NOP,PG,2,2,24.0,3.5,7.0,0.500,0.0,0.5,0.000,3.5,6.5,0.538,0.500,0.5,1.0,0.500,0.5,6.0,3.0,7.5,100.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,1.0,28.0,0.0,0.0,10.2,0.929,0.071,0.643,0.143,0.071,0.071,0.538,0.000,0.556,0.500,1.000,0.000,0.143,0.000,0.000,0.0,Guards,0
235,Andrew Nembhard,25,IND,SG,7,7,26.7,2.9,7.4,0.385,0.4,1.9,0.231,2.4,5.6,0.436,0.413,1.1,1.4,0.800,0.1,4.7,1.4,7.3,4.0,96.0,0.0,0.0,0.0,7.0,2.0,1.0,5.0,80.0,0.0,3.0,13.5,0.750,0.192,0.115,0.288,0.154,0.250,0.436,0.900,0.167,0.333,0.250,0.231,0.412,1.000,0.000,0.0,Guards,0
294,Lonzo Ball,27,CHI,PG,3,0,15.7,1.7,4.7,0.357,1.3,4.0,0.333,0.3,0.7,0.500,0.500,0.0,0.0,0.000,0.0,3.7,1.0,4.7,100.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,29.0,0.0,0.0,23.0,0.143,0.071,0.071,0.000,0.000,0.857,0.500,1.000,0.000,0.000,0.000,0.333,1.000,1.000,0.000,0.0,Guards,0
232,Marcus Smart,30,MEM,PG,7,5,20.1,2.1,7.1,0.300,0.9,4.0,0.214,1.3,3.1,0.409,0.360,2.3,2.7,0.842,1.0,3.7,2.0,7.4,32.0,63.0,5.0,0.0,0.0,9.0,5.0,0.0,5.0,62.0,0.0,3.0,17.3,0.440,0.080,0.320,0.020,0.020,0.560,0.409,0.750,0.313,0.000,1.000,0.214,0.222,1.000,0.000,0.0,Guards,0
215,Cameron Payne,30,NYK,PG,11,0,17.1,3.1,7.0,0.442,1.8,4.1,0.444,1.3,2.9,0.438,0.571,0.2,0.4,0.500,0.5,3.4,0.6,8.2,57.0,43.0,0.0,0.0,0.0,4.0,1.0,1.0,1.0,89.0,1.0,1.0,18.0,0.416,0.104,0.195,0.078,0.039,0.584,0.438,0.875,0.267,0.333,0.333,0.444,0.286,0.650,0.000,0.0,Guards,0
260,Kyle Lowry,38,PHI,PG,13,10,24.2,1.8,4.8,0.381,1.3,3.7,0.354,0.5,1.2,0.467,0.516,1.2,1.5,0.789,0.5,3.4,0.8,6.2,100.0,0.0,0.0,0.0,0.0,8.0,1.0,0.0,5.0,108.0,0.0,3.0,22.1,0.238,0.048,0.127,0.000,0.063,0.762,0.467,1.000,0.375,0.000,0.250,0.354,0.571,0.941,0.000,0.0,Guards,0
179,Donte DiVincenzo,28,MIN,SG,15,0,25.9,3.2,9.2,0.348,2.3,7.2,0.324,0.9,2.0,0.433,0.475,0.7,0.9,0.786,0.4,3.3,1.7,9.5,66.0,34.0,0.0,0.0,0.0,15.0,10.0,1.0,4.0,120.0,0.0,7.0,21.0,0.217,0.130,0.072,0.014,0.000,0.783,0.433,0.500,0.300,0.500,0.000,0.324,0.462,0.743,0.014,1.0,Guards,0
227,Kris Dunn,30,LAC,PG,17,6,22.6,3.1,6.4,0.486,1.2,3.4,0.368,1.9,3.1,0.615,0.583,0.2,0.5,0.375,0.9,3.1,1.1,7.6,100.0,0.0,0.0,0.0,0.0,7.0,5.0,3.0,4.0,128.0,2.0,0.0,16.0,0.477,0.147,0.248,0.055,0.028,0.523,0.615,0.688,0.556,0.667,0.667,0.368,0.531,0.810,0.018,2.0,Guards,0
282,Vince Williams Jr.,24,MEM,SG,3,0,15.0,2.3,6.3,0.368,0.3,2.7,0.125,2.0,3.7,0.545,0.395,0.3,1.3,0.250,1.3,3.0,2.3,5.3,48.0,51.0,1.0,0.0,0.0,3.0,4.0,0.0,0.0,20.0,0.0,0.0,14.2,0.579,0.263,0.316,0.000,0.000,0.421,0.545,0.600,0.500,0.000,0.000,0.125,0.167,1.000,0.053,1.0,Guards,0
117,Malik Monk,26,SAC,SG,10,0,25.0,4.6,10.4,0.442,1.2,4.3,0.279,3.4,6.1,0.557,0.500,2.2,2.6,0.846,0.0,2.8,0.8,12.6,32.0,67.0,1.0,0.0,0.0,4.0,2.0,1.0,12.0,65.0,2.0,7.0,15.2,0.587,0.163,0.260,0.115,0.048,0.413,0.557,0.529,0.630,0.417,0.600,0.279,0.441,0.750,0.048,4.0,Guards,0


In [30]:
print("\nCatch and Shoot Wings:")
Swings_set


Catch and Shoot Wings:


,Player,Age,Team,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,AST,TOV,PTS,PG%,SG%,SF%,PF%,C%,BadPassTO,LostBallTO,OffFoulCmtd,ShootFoulDrawn,PtsGenbyAsts,And1s,FGA_Blkd,AvgFGDist,%FGA_2P,%FGA_0-3,%FGA_3-10,%FGA_10-16,%FGA_16-3P,%FGA_3P,FG%_2P,FG%_0-3,FG%_3-10,FG%_10-16,FG%_16-3P,FG%_3P,%FG_Astd_2P,%FG_Astd_3P,%FGA_Dunks,Num_Dunks,Position Group,%FGA_Astd,Cluster
109,Klay Thompson,34,DAL,SF,16,16,28.4,4.7,12.3,0.381,2.9,8.2,0.359,1.8,4.1,0.424,0.500,0.8,0.8,1.000,0.6,1.9,1.0,13.1,0.0,4.0,84.0,12.0,0.0,11.0,3.0,1.0,5.0,69.0,1.0,6.0,20.5,0.335,0.030,0.091,0.132,0.081,0.665,0.424,0.667,0.278,0.462,0.438,0.359,0.750,0.979,0.000,0.0,Wings,1.729,0
111,Santi Aldama,24,MEM,PF,16,11,28.6,5.3,11.0,0.477,1.4,4.8,0.289,3.9,6.3,0.620,0.540,1.0,1.1,0.889,1.5,3.4,1.5,12.9,0.0,0.0,20.0,69.0,11.0,16.0,5.0,1.0,9.0,128.0,2.0,5.0,13.2,0.568,0.273,0.250,0.034,0.011,0.432,0.620,0.792,0.477,0.500,0.000,0.289,0.742,0.955,0.131,23.0,Wings,1.697,0
118,Keegan Murray,24,SAC,PF,16,16,36.9,4.9,11.4,0.426,1.8,5.8,0.304,3.1,5.7,0.549,0.503,1.1,1.3,0.857,2.3,1.8,1.1,12.6,0.0,0.0,16.0,83.0,0.0,9.0,6.0,1.0,11.0,70.0,2.0,5.0,16.6,0.497,0.137,0.191,0.109,0.060,0.503,0.549,0.920,0.400,0.400,0.455,0.304,0.640,1.000,0.082,14.0,Wings,1.640,0
124,Ochai Agbaji,24,TOR,SG,16,14,32.1,4.9,9.4,0.520,1.8,4.1,0.446,3.1,5.3,0.576,0.617,0.9,1.3,0.700,1.1,2.1,1.1,12.4,3.0,87.0,9.0,0.0,0.0,6.0,6.0,2.0,11.0,85.0,4.0,11.0,12.2,0.567,0.373,0.160,0.033,0.000,0.433,0.576,0.643,0.500,0.200,0.000,0.446,0.735,0.931,0.053,8.0,Wings,1.666,0
125,Tari Eason,23,HOU,SF,17,0,23.0,5.0,9.4,0.531,1.1,3.0,0.353,3.9,6.4,0.615,0.588,1.2,1.7,0.724,2.1,1.2,0.9,12.3,0.0,0.0,22.0,74.0,5.0,5.0,4.0,3.0,11.0,49.0,2.0,7.0,11.2,0.681,0.400,0.175,0.081,0.025,0.319,0.615,0.766,0.357,0.538,0.250,0.353,0.493,1.000,0.125,17.0,Wings,1.493,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,Pat Connaughton,32,MIL,SG,13,0,17.6,1.7,4.8,0.355,0.9,3.1,0.300,0.8,1.7,0.455,0.452,0.6,0.8,0.800,0.5,1.8,0.8,4.9,2.0,37.0,60.0,2.0,0.0,9.0,1.0,0.0,3.0,56.0,0.0,5.0,17.7,0.355,0.210,0.129,0.016,0.000,0.645,0.455,0.615,0.250,0.000,0.000,0.300,1.000,0.917,0.016,1.0,Wings,1.917,0
290,Pelle Larsson,23,MIA,SG,8,0,13.1,1.8,3.3,0.538,0.6,1.5,0.417,1.1,1.8,0.643,0.635,0.8,1.3,0.600,0.3,1.4,0.5,4.9,14.0,39.0,45.0,3.0,0.0,2.0,1.0,0.0,5.0,29.0,2.0,1.0,13.3,0.538,0.269,0.269,0.000,0.000,0.462,0.643,0.714,0.571,0.000,0.000,0.417,0.778,1.000,0.038,1.0,Wings,1.778,0
292,Dalen Terry,22,CHI,SG,17,0,13.8,1.6,3.2,0.491,0.5,1.1,0.444,1.1,2.2,0.514,0.564,1.2,1.6,0.778,0.8,1.1,1.0,4.9,2.0,21.0,72.0,5.0,0.0,9.0,3.0,0.0,12.0,48.0,1.0,4.0,10.0,0.673,0.527,0.109,0.000,0.036,0.327,0.514,0.552,0.333,0.000,0.500,0.444,0.579,1.000,0.055,3.0,Wings,1.579,0
295,Jamal Cain,25,NOP,SF,12,0,15.3,1.6,3.7,0.432,0.8,2.0,0.375,0.8,1.7,0.500,0.534,0.8,1.2,0.643,0.9,0.8,0.6,4.7,0.0,30.0,52.0,17.0,0.0,5.0,0.0,1.0,7.0,22.0,2.0,0.0,15.7,0.455,0.205,0.182,0.045,0.023,0.545,0.500,0.778,0.375,0.000,0.000,0.375,0.800,1.000,0.000,0.0,Wings,1.800,0


In [31]:
print("\nBalanced Wings:")
Swings_bal


Balanced Wings:


,Player,Age,Team,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,AST,TOV,PTS,PG%,SG%,SF%,PF%,C%,BadPassTO,LostBallTO,OffFoulCmtd,ShootFoulDrawn,PtsGenbyAsts,And1s,FGA_Blkd,AvgFGDist,%FGA_2P,%FGA_0-3,%FGA_3-10,%FGA_10-16,%FGA_16-3P,%FGA_3P,FG%_2P,FG%_0-3,FG%_3-10,FG%_10-16,FG%_16-3P,FG%_3P,%FG_Astd_2P,%FG_Astd_3P,%FGA_Dunks,Num_Dunks,Position Group,%FGA_Astd,Cluster
24,Norman Powell,31,LAC,SG,15,15,33.4,8.1,16.5,0.490,3.9,7.9,0.487,4.2,8.5,0.492,0.607,3.3,4.0,0.817,0.3,2.3,2.1,23.3,3.0,69.0,28.0,0.0,0.0,9.0,18.0,2.0,24.0,87.0,6.0,14.0,16.2,0.518,0.162,0.194,0.121,0.040,0.482,0.492,0.650,0.438,0.433,0.300,0.487,0.619,0.741,0.008,2.0,Wings,1.360,2
26,Brandon Ingram,27,NOP,SF,16,16,33.7,9.0,19.4,0.465,2.4,6.7,0.364,6.6,12.7,0.517,0.527,2.4,2.9,0.830,1.0,5.4,3.9,22.9,0.0,7.0,64.0,29.0,0.0,26.0,23.0,11.0,23.0,209.0,4.0,20.0,16.0,0.655,0.119,0.155,0.229,0.152,0.345,0.517,0.730,0.417,0.549,0.404,0.364,0.419,0.821,0.026,8.0,Wings,1.240,2
32,Zach LaVine,29,CHI,SF,14,14,34.1,8.1,15.6,0.521,3.2,7.4,0.437,4.9,8.3,0.595,0.623,2.8,3.5,0.796,0.2,4.2,3.2,22.3,0.0,6.0,81.0,13.0,0.0,20.0,20.0,1.0,21.0,143.0,5.0,9.0,16.5,0.530,0.260,0.068,0.073,0.128,0.470,0.595,0.667,0.400,0.625,0.536,0.437,0.449,0.600,0.059,13.0,Wings,1.049,2
34,Jalen Williams,23,OKC,SG,16,16,32.6,8.8,16.9,0.517,2.0,5.1,0.390,6.8,11.8,0.571,0.576,2.3,2.8,0.818,1.1,5.2,2.1,21.8,0.0,0.0,5.0,64.0,31.0,16.0,15.0,3.0,25.0,204.0,8.0,12.0,12.7,0.697,0.277,0.188,0.162,0.070,0.303,0.571,0.747,0.373,0.545,0.474,0.390,0.463,0.813,0.100,23.0,Wings,1.276,2
37,Pascal Siakam,30,IND,PF,16,16,33.9,8.1,14.6,0.554,1.9,4.3,0.449,6.1,10.3,0.598,0.620,2.8,3.6,0.772,1.8,3.6,1.4,20.8,0.0,0.0,21.0,75.0,4.0,7.0,10.0,4.0,30.0,135.0,9.0,9.0,12.8,0.704,0.223,0.215,0.197,0.069,0.296,0.598,0.808,0.560,0.478,0.375,0.449,0.490,1.000,0.021,5.0,Wings,1.490,2
45,Jalen Johnson,23,ATL,SF,16,16,36.5,7.5,15.8,0.474,1.6,4.6,0.338,5.9,11.2,0.531,0.524,2.9,3.9,0.730,2.1,5.3,3.3,19.4,0.0,0.0,20.0,80.0,0.0,26.0,8.0,6.0,30.0,198.0,5.0,23.0,11.3,0.708,0.285,0.285,0.123,0.016,0.292,0.531,0.750,0.417,0.258,0.750,0.338,0.621,0.920,0.079,18.0,Wings,1.541,2
46,Cameron Johnson,28,BRK,PF,16,16,33.6,6.5,13.1,0.498,3.4,8.1,0.423,3.1,4.9,0.620,0.629,2.8,3.1,0.918,0.6,2.9,0.8,19.3,0.0,2.0,13.0,59.0,27.0,5.0,5.0,1.0,16.0,111.0,6.0,6.0,18.2,0.378,0.201,0.086,0.048,0.043,0.622,0.620,0.786,0.444,0.500,0.333,0.423,0.653,0.909,0.038,7.0,Wings,1.562,2
48,Bennedict Mathurin,22,IND,SG,16,11,30.9,6.2,12.1,0.510,1.8,3.9,0.452,4.4,8.3,0.538,0.582,4.8,5.7,0.835,1.3,1.6,2.1,18.9,0.0,25.0,75.0,0.0,0.0,11.0,13.0,7.0,43.0,61.0,10.0,12.0,12.6,0.680,0.335,0.186,0.057,0.103,0.320,0.538,0.662,0.500,0.455,0.250,0.452,0.521,0.714,0.026,5.0,Wings,1.235,2
52,Brandon Miller,22,CHO,SF,11,11,32.1,6.7,16.3,0.413,3.5,9.6,0.368,3.2,6.6,0.479,0.522,1.6,1.7,0.947,0.6,3.8,2.4,18.6,0.0,0.0,89.0,11.0,0.0,11.0,11.0,3.0,10.0,105.0,3.0,10.0,18.7,0.408,0.123,0.162,0.095,0.028,0.592,0.479,0.727,0.345,0.471,0.200,0.368,0.600,0.872,0.050,8.0,Wings,1.472,2
53,Derrick White,30,BOS,SG,16,16,34.9,6.4,13.8,0.464,3.7,9.1,0.404,2.7,4.6,0.581,0.598,1.9,2.3,0.861,1.1,4.3,1.5,18.4,7.0,83.0,10.0,0.0,0.0,21.0,2.0,1.0,14.0,170.0,3.0,9.0,19.6,0.336,0.091,0.164,0.050,0.032,0.664,0.581,0.550,0.583,0.818,0.286,0.404,0.512,0.898,0.009,2.0,Wings,1.410,2


In [32]:
print("\nIso Scoring Wings:")
Swings_iso


Iso Scoring Wings:


,Player,Age,Team,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,AST,TOV,PTS,PG%,SG%,SF%,PF%,C%,BadPassTO,LostBallTO,OffFoulCmtd,ShootFoulDrawn,PtsGenbyAsts,And1s,FGA_Blkd,AvgFGDist,%FGA_2P,%FGA_0-3,%FGA_3-10,%FGA_10-16,%FGA_16-3P,%FGA_3P,FG%_2P,FG%_0-3,FG%_3-10,FG%_10-16,FG%_16-3P,FG%_3P,%FG_Astd_2P,%FG_Astd_3P,%FGA_Dunks,Num_Dunks,Position Group,%FGA_Astd,Cluster
0,Giannis Antetokounmpo,30,MIL,PF,15,15,35.3,13.1,21.4,0.614,0.1,0.7,0.182,13.0,20.7,0.629,0.617,6.0,10.1,0.596,2.1,6.4,3.2,32.4,0.0,0.0,0.0,63.0,37.0,15.0,16.0,11.0,84.0,254.0,27.0,19.0,6.5,0.966,0.570,0.159,0.072,0.165,0.034,0.629,0.765,0.373,0.478,0.472,0.182,0.379,0.000,0.190,59.0,Wings,0.379,1
3,Jayson Tatum,26,BOS,PF,16,16,36.6,9.1,20.1,0.455,4.1,10.9,0.371,5.1,9.1,0.555,0.556,6.8,8.4,0.806,0.4,6.1,2.9,29.1,0.0,2.0,48.0,51.0,0.0,23.0,15.0,6.0,45.0,256.0,8.0,14.0,17.9,0.455,0.150,0.150,0.078,0.078,0.545,0.555,0.875,0.417,0.320,0.440,0.371,0.333,0.492,0.050,15.0,Wings,0.825,1
7,Shai Gilgeous-Alexander,26,OKC,SG,16,16,34.1,9.9,19.9,0.500,1.8,5.4,0.337,8.1,14.5,0.560,0.546,7.0,7.8,0.896,0.9,6.3,2.6,28.7,0.0,11.0,59.0,28.0,2.0,25.0,10.0,0.0,52.0,254.0,10.0,12.0,12.1,0.730,0.258,0.252,0.167,0.053,0.270,0.560,0.768,0.400,0.528,0.412,0.337,0.208,0.207,0.022,7.0,Wings,0.415,1
9,Anthony Edwards,23,MIN,SG,15,15,37.1,9.9,20.9,0.475,4.8,11.1,0.431,5.1,9.8,0.524,0.589,3.3,4.3,0.766,0.5,3.7,3.1,27.9,2.0,69.0,28.0,0.0,0.0,20.0,16.0,3.0,35.0,133.0,11.0,10.0,17.6,0.468,0.159,0.156,0.061,0.092,0.532,0.524,0.640,0.531,0.316,0.448,0.431,0.312,0.500,0.029,8.0,Wings,0.812,1
12,Jaylen Brown,28,BOS,SF,12,12,36.9,8.7,20.3,0.426,2.1,7.3,0.284,6.6,13.0,0.506,0.477,5.9,7.7,0.772,1.6,4.4,2.5,25.3,0.0,16.0,83.0,1.0,0.0,7.0,14.0,9.0,40.0,134.0,4.0,13.0,14.4,0.639,0.164,0.262,0.131,0.082,0.361,0.506,0.675,0.453,0.438,0.450,0.284,0.316,0.680,0.037,8.0,Wings,0.996,1
18,Cam Thomas,23,BRK,SG,15,15,33.9,8.0,17.6,0.455,2.8,7.3,0.385,5.2,10.3,0.503,0.534,5.4,6.2,0.871,0.4,3.3,3.1,24.2,8.0,64.0,28.0,0.0,0.0,15.0,22.0,5.0,44.0,121.0,11.0,21.0,15.7,0.587,0.167,0.178,0.136,0.106,0.413,0.503,0.614,0.383,0.500,0.536,0.385,0.538,0.571,0.000,0.0,Wings,1.109,1
19,Devin Booker,28,PHO,SG,16,16,36.8,7.9,18.0,0.438,2.6,7.6,0.339,5.3,10.4,0.509,0.509,5.8,6.6,0.886,1.3,6.4,2.1,24.1,1.0,37.0,56.0,7.0,0.0,11.0,13.0,4.0,39.0,263.0,5.0,18.0,16.6,0.580,0.101,0.177,0.160,0.142,0.420,0.509,0.690,0.471,0.457,0.488,0.339,0.435,0.707,0.010,2.0,Wings,1.142,1
20,LeBron James,40,LAL,SF,15,15,35.4,8.8,17.0,0.518,2.5,5.7,0.430,6.3,11.3,0.562,0.590,3.9,5.4,0.728,0.8,9.3,3.7,24.0,0.0,0.0,46.0,53.0,0.0,37.0,11.0,3.0,37.0,325.0,13.0,14.0,13.4,0.663,0.278,0.227,0.078,0.078,0.337,0.562,0.704,0.500,0.500,0.300,0.430,0.411,0.649,0.051,13.0,Wings,1.060,1
21,Kyrie Irving,32,DAL,SG,15,15,34.8,9.0,16.9,0.531,2.9,5.6,0.512,6.1,11.3,0.541,0.616,3.1,3.5,0.868,1.1,5.3,1.9,23.9,20.0,74.0,6.0,0.0,0.0,15.0,11.0,2.0,24.0,184.0,7.0,9.0,14.3,0.669,0.165,0.272,0.150,0.083,0.331,0.541,0.690,0.522,0.500,0.381,0.512,0.283,0.581,0.000,0.0,Wings,0.864,1
22,RJ Barrett,24,TOR,SG,13,13,34.2,8.5,19.3,0.438,2.0,5.9,0.338,6.5,13.4,0.483,0.490,4.8,6.8,0.716,0.9,6.2,3.3,23.8,0.0,18.0,30.0,52.0,0.0,21.0,18.0,1.0,45.0,187.0,9.0,22.0,11.4,0.693,0.299,0.327,0.044,0.024,0.307,0.483,0.653,0.341,0.455,0.333,0.338,0.298,0.769,0.028,6.0,Wings,1.067,1


In [33]:
print("\nBack to Basket Bigs:")
Sbigs_b2b


Back to Basket Bigs:


,Player,Age,Team,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,AST,TOV,PTS,PG%,SG%,SF%,PF%,C%,BadPassTO,LostBallTO,OffFoulCmtd,ShootFoulDrawn,PtsGenbyAsts,And1s,FGA_Blkd,AvgFGDist,%FGA_2P,%FGA_0-3,%FGA_3-10,%FGA_10-16,%FGA_16-3P,%FGA_3P,FG%_2P,FG%_0-3,FG%_3-10,FG%_10-16,FG%_16-3P,FG%_3P,%FG_Astd_2P,%FG_Astd_3P,%FGA_Dunks,Num_Dunks,Position Group,%FGA_0-10,Cluster
94,Ivica Zubac,27,LAC,C,17,17,33.7,6.6,11.4,0.577,0.0,0.0,0.000,6.6,11.4,0.577,0.577,1.9,3.4,0.579,4.2,2.5,2.4,15.1,0.0,0.0,0.0,0.0,100.0,10.0,15.0,8.0,29.0,101.0,9.0,14.0,4.3,1.000,0.448,0.474,0.067,0.010,0.000,0.577,0.655,0.543,0.308,0.5,0.000,0.750,0.0,0.186,32.0,Bigs,0.922,1
98,Jarrett Allen,26,CLE,C,17,17,29.5,5.8,8.4,0.685,0.0,0.0,0.000,5.8,8.4,0.685,0.685,2.8,3.7,0.762,2.3,1.8,1.1,14.4,0.0,0.0,0.0,0.0,100.0,9.0,5.0,2.0,27.0,74.0,7.0,9.0,3.2,1.000,0.664,0.273,0.042,0.021,0.000,0.685,0.779,0.590,0.167,0.0,0.000,0.837,0.0,0.343,44.0,Bigs,0.937,1
116,Daniel Gafford,26,DAL,C,16,11,20.9,5.3,7.5,0.708,0.0,0.0,0.000,5.3,7.5,0.708,0.708,1.9,2.9,0.674,3.0,1.0,0.9,12.6,0.0,0.0,0.0,1.0,99.0,3.0,6.0,4.0,20.0,39.0,6.0,10.0,2.4,1.000,0.683,0.308,0.008,0.000,0.000,0.708,0.744,0.622,1.000,0.0,0.000,0.729,0.0,0.308,32.0,Bigs,0.991,1
132,Clint Capela,30,ATL,C,17,17,22.7,5.1,8.2,0.621,0.0,0.0,0.000,5.1,8.2,0.621,0.621,1.4,2.6,0.545,3.4,1.3,0.9,11.6,0.0,0.0,0.0,0.0,100.0,4.0,3.0,6.0,22.0,53.0,4.0,17.0,2.7,1.000,0.664,0.307,0.029,0.000,0.000,0.621,0.677,0.512,0.500,0.0,0.000,0.782,0.0,0.300,37.0,Bigs,0.971,1
143,Zach Edey,22,MEM,C,14,9,20.1,4.6,7.5,0.610,0.4,0.7,0.600,4.1,6.8,0.611,0.638,1.6,2.4,0.667,3.1,0.8,1.6,11.1,0.0,0.0,0.0,0.0,100.0,5.0,8.0,7.0,16.0,29.0,4.0,9.0,5.7,0.905,0.505,0.362,0.029,0.010,0.095,0.611,0.811,0.316,0.667,1.0,0.600,0.690,1.0,0.276,28.0,Bigs,0.867,1
146,Rudy Gobert,32,MIN,C,15,15,33.1,4.1,6.5,0.639,0.0,0.0,0.000,4.1,6.5,0.639,0.639,2.7,3.4,0.804,3.2,1.9,1.9,11.0,0.0,0.0,0.0,0.0,100.0,6.0,11.0,9.0,24.0,75.0,6.0,16.0,2.5,1.000,0.680,0.278,0.041,0.000,0.000,0.639,0.773,0.407,0.000,0.0,0.000,0.774,0.0,0.474,39.0,Bigs,0.958,1
148,Nick Richards,27,CHO,C,5,5,27.4,4.2,6.6,0.636,0.0,0.2,0.000,4.2,6.4,0.656,0.636,2.6,4.4,0.591,4.0,1.8,1.4,11.0,0.0,0.0,0.0,0.0,100.0,2.0,1.0,2.0,13.0,23.0,4.0,5.0,3.1,0.970,0.758,0.212,0.000,0.000,0.030,0.656,0.720,0.429,0.000,0.0,0.000,0.524,0.0,0.242,7.0,Bigs,0.970,1
157,Onyeka Okongwu,24,ATL,C,15,0,23.5,4.1,7.7,0.526,0.4,1.9,0.207,3.7,5.8,0.632,0.552,2.1,2.7,0.780,2.0,1.5,0.9,10.7,0.0,0.0,0.0,0.0,100.0,4.0,6.0,3.0,15.0,55.0,3.0,11.0,8.1,0.750,0.517,0.181,0.052,0.000,0.250,0.632,0.700,0.429,0.667,0.0,0.207,0.782,1.0,0.207,22.0,Bigs,0.698,1
167,Robert Williams,27,POR,C,6,0,20.0,4.3,5.7,0.765,0.2,0.3,0.500,4.2,5.3,0.781,0.779,1.3,1.5,0.889,2.0,1.5,1.2,10.2,0.0,0.0,0.0,9.0,91.0,5.0,1.0,1.0,5.0,22.0,3.0,2.0,3.1,0.941,0.765,0.176,0.000,0.000,0.059,0.781,0.846,0.500,0.000,0.0,0.500,0.760,1.0,0.324,10.0,Bigs,0.941,1
185,Walker Kessler,23,UTA,C,9,9,26.4,3.9,5.7,0.686,0.0,0.0,0.000,3.9,5.7,0.686,0.686,1.4,2.6,0.565,3.8,1.0,1.7,9.2,0.0,0.0,0.0,0.0,100.0,4.0,6.0,3.0,12.0,21.0,3.0,7.0,2.6,1.000,0.667,0.294,0.020,0.020,0.000,0.686,0.853,0.400,0.000,0.0,0.000,0.686,0.0,0.333,17.0,Bigs,0.961,1


In [34]:
print("\nBalanced Bigs:")
Sbigs_bal


Balanced Bigs:


,Player,Age,Team,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,AST,TOV,PTS,PG%,SG%,SF%,PF%,C%,BadPassTO,LostBallTO,OffFoulCmtd,ShootFoulDrawn,PtsGenbyAsts,And1s,FGA_Blkd,AvgFGDist,%FGA_2P,%FGA_0-3,%FGA_3-10,%FGA_10-16,%FGA_16-3P,%FGA_3P,FG%_2P,FG%_0-3,FG%_3-10,FG%_10-16,FG%_16-3P,FG%_3P,%FG_Astd_2P,%FG_Astd_3P,%FGA_Dunks,Num_Dunks,Position Group,%FGA_0-10,Cluster
1,Anthony Davis,31,LAL,C,14,14,35.2,10.9,19.1,0.573,0.9,2.1,0.433,10.0,16.9,0.591,0.597,8.5,10.9,0.783,2.6,2.8,2.1,31.3,0.0,0.0,0.0,6.0,94.0,14.0,10.0,2.0,60.0,95.0,15.0,14.0,8.2,0.888,0.378,0.326,0.101,0.082,0.112,0.591,0.782,0.506,0.333,0.364,0.433,0.614,1.000,0.127,31.0,Bigs,0.704,2
2,Nikola Jokić,29,DEN,C,11,11,38.3,11.0,19.5,0.565,2.2,3.7,0.585,8.8,15.7,0.561,0.621,5.8,7.0,0.831,4.6,11.5,4.0,30.0,0.0,0.0,0.0,0.0,100.0,22.0,17.0,2.0,28.0,303.0,3.0,14.0,9.2,0.808,0.280,0.453,0.065,0.009,0.192,0.561,0.617,0.546,0.357,1.000,0.585,0.577,0.875,0.014,2.0,Bigs,0.733,2
4,Paolo Banchero,22,ORL,PF,5,5,36.4,9.6,19.4,0.495,2.2,6.4,0.344,7.4,13.0,0.569,0.552,7.6,11.8,0.644,2.4,5.6,2.2,29.0,0.0,0.0,0.0,98.0,2.0,6.0,4.0,0.0,29.0,70.0,5.0,6.0,12.9,0.670,0.340,0.134,0.113,0.082,0.330,0.569,0.697,0.385,0.364,0.625,0.344,0.486,0.545,0.041,4.0,Bigs,0.474,2
11,Karl-Anthony Towns,29,NYK,C,14,14,33.0,9.6,17.5,0.551,2.6,5.3,0.500,7.0,12.2,0.573,0.627,4.9,5.7,0.850,2.8,2.9,1.9,26.8,0.0,0.0,0.0,0.0,100.0,8.0,7.0,8.0,43.0,93.0,14.0,18.0,11.2,0.698,0.343,0.278,0.061,0.016,0.302,0.573,0.631,0.559,0.467,0.000,0.500,0.541,0.973,0.073,17.0,Bigs,0.621,2
27,Jaren Jackson Jr.,25,MEM,C,14,14,28.9,8.2,15.6,0.528,1.9,5.2,0.356,6.4,10.4,0.614,0.587,4.6,5.8,0.802,1.3,1.1,2.3,22.9,0.0,0.0,0.0,47.0,53.0,9.0,13.0,5.0,33.0,37.0,9.0,13.0,12.5,0.665,0.156,0.454,0.055,0.000,0.335,0.614,0.794,0.556,0.583,0.000,0.356,0.449,0.885,0.055,12.0,Bigs,0.610,2
30,Zion Williamson,24,NOP,PF,6,6,31.0,7.8,17.3,0.452,0.3,1.0,0.333,7.5,16.3,0.459,0.462,6.7,10.0,0.667,2.7,5.3,3.7,22.7,0.0,0.0,2.0,85.0,13.0,10.0,4.0,8.0,32.0,80.0,8.0,19.0,5.7,0.942,0.394,0.500,0.038,0.010,0.058,0.459,0.659,0.346,0.000,0.000,0.333,0.556,0.500,0.077,7.0,Bigs,0.894,2
35,Julius Randle,30,MIN,PF,15,15,34.3,7.3,14.6,0.502,1.8,4.9,0.365,5.5,9.7,0.572,0.564,5.3,6.5,0.814,1.9,4.3,3.0,21.7,0.0,0.0,1.0,96.0,3.0,13.0,18.0,9.0,44.0,167.0,8.0,17.0,12.2,0.662,0.251,0.260,0.119,0.032,0.338,0.572,0.782,0.474,0.346,0.571,0.365,0.446,0.815,0.064,14.0,Bigs,0.511,2
41,Domantas Sabonis,28,SAC,C,14,14,36.7,7.8,12.4,0.630,1.0,2.5,0.400,6.8,9.9,0.688,0.671,3.9,4.5,0.857,2.8,6.4,3.6,20.4,0.0,0.0,0.0,2.0,98.0,15.0,24.0,9.0,27.0,211.0,5.0,5.0,9.1,0.798,0.410,0.260,0.064,0.064,0.202,0.688,0.746,0.644,0.545,0.636,0.400,0.600,1.000,0.110,18.0,Bigs,0.670,2
42,Nikola Vučević,34,CHI,C,17,17,32.4,8.2,14.2,0.577,2.1,4.5,0.461,6.1,9.7,0.630,0.649,1.8,2.1,0.857,2.1,3.2,1.8,20.2,0.0,0.0,0.0,0.0,100.0,13.0,8.0,4.0,17.0,141.0,5.0,7.0,12.3,0.685,0.216,0.332,0.104,0.033,0.315,0.630,0.750,0.613,0.440,0.625,0.461,0.750,0.971,0.012,3.0,Bigs,0.548,2
56,Alperen Sengun,22,HOU,C,17,17,29.2,7.2,14.8,0.484,0.5,1.6,0.296,6.7,13.2,0.507,0.500,3.3,4.4,0.747,3.5,4.6,2.1,18.1,0.0,0.0,0.0,0.0,100.0,14.0,12.0,6.0,34.0,189.0,11.0,27.0,7.7,0.893,0.345,0.369,0.147,0.032,0.107,0.507,0.563,0.473,0.541,0.125,0.296,0.447,1.000,0.063,12.0,Bigs,0.714,2


In [35]:
print("\nStretch Bigs:")
Sbigs_stretch


Stretch Bigs:


,Player,Age,Team,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,AST,TOV,PTS,PG%,SG%,SF%,PF%,C%,BadPassTO,LostBallTO,OffFoulCmtd,ShootFoulDrawn,PtsGenbyAsts,And1s,FGA_Blkd,AvgFGDist,%FGA_2P,%FGA_0-3,%FGA_3-10,%FGA_10-16,%FGA_16-3P,%FGA_3P,FG%_2P,FG%_0-3,FG%_3-10,FG%_10-16,FG%_16-3P,FG%_3P,%FG_Astd_2P,%FG_Astd_3P,%FGA_Dunks,Num_Dunks,Position Group,%FGA_0-10,Cluster
10,Kevin Durant,36,PHO,PF,9,9,38.8,9.9,17.9,0.553,2.7,6.2,0.429,7.2,11.7,0.619,0.627,5.1,6.1,0.836,0.2,3.4,3.3,27.6,0.0,0.0,0.0,84.0,16.0,17.0,6.0,1.0,26.0,76.0,9.0,3.0,15.0,0.652,0.155,0.130,0.292,0.075,0.348,0.619,0.920,0.667,0.468,0.500,0.429,0.492,0.750,0.037,6.0,Bigs,0.285,0
29,Victor Wembanyama,21,SAS,C,13,13,31.5,8.3,17.6,0.472,2.9,8.6,0.339,5.4,9.0,0.598,0.555,3.2,3.6,0.872,2.3,2.8,3.5,22.7,0.0,0.0,0.0,0.0,100.0,14.0,24.0,3.0,21.0,87.0,5.0,8.0,16.9,0.511,0.236,0.096,0.066,0.114,0.489,0.598,0.833,0.455,0.467,0.308,0.339,0.614,0.789,0.140,28.0,Bigs,0.332,0
43,Joel Embiid,30,PHI,C,4,4,31.3,5.5,14.5,0.379,0.8,4.5,0.167,4.8,10.0,0.475,0.405,8.0,8.0,1.000,1.3,3.5,3.3,19.8,0.0,0.0,0.0,0.0,100.0,5.0,5.0,3.0,11.0,34.0,0.0,3.0,15.4,0.690,0.138,0.155,0.241,0.155,0.310,0.475,0.625,0.667,0.429,0.222,0.167,0.526,1.000,0.017,1.0,Bigs,0.293,0
50,Lauri Markkanen,27,UTA,PF,12,12,31.4,5.7,12.6,0.450,2.7,6.3,0.427,3.0,6.3,0.474,0.556,4.8,5.5,0.864,1.8,2.3,1.4,18.8,0.0,0.0,0.0,76.0,24.0,6.0,7.0,3.0,24.0,67.0,7.0,8.0,16.1,0.503,0.205,0.185,0.046,0.066,0.497,0.474,0.516,0.571,0.286,0.200,0.427,0.583,1.000,0.086,10.0,Bigs,0.390,0
70,OG Anunoby,27,NYK,PF,15,15,36.3,6.3,12.6,0.503,2.1,5.5,0.373,4.3,7.1,0.604,0.585,2.4,2.9,0.818,1.1,2.1,1.2,17.1,0.0,0.0,0.0,100.0,0.0,8.0,5.0,2.0,22.0,83.0,4.0,12.0,13.5,0.561,0.349,0.111,0.074,0.026,0.439,0.604,0.742,0.333,0.357,0.600,0.373,0.797,1.000,0.159,27.0,Bigs,0.460,0
72,Myles Turner,28,IND,C,14,14,32.6,6.1,12.6,0.480,2.2,5.1,0.431,3.9,7.5,0.514,0.568,2.5,3.6,0.700,0.9,1.4,2.7,16.9,0.0,0.0,0.0,0.0,100.0,13.0,10.0,10.0,29.0,44.0,9.0,7.0,14.4,0.593,0.232,0.198,0.096,0.068,0.407,0.514,0.732,0.343,0.471,0.333,0.431,0.778,0.968,0.096,15.0,Bigs,0.430,0
75,Chet Holmgren,22,OKC,C,10,10,26.5,5.6,11.1,0.505,1.4,3.7,0.378,4.2,7.4,0.568,0.568,3.8,4.9,0.776,1.6,2.0,2.3,16.4,0.0,0.0,0.0,0.0,100.0,3.0,15.0,3.0,24.0,43.0,6.0,6.0,11.8,0.667,0.360,0.171,0.108,0.027,0.333,0.568,0.750,0.211,0.417,1.000,0.378,0.667,1.000,0.153,17.0,Bigs,0.531,0
95,Paul George,34,PHI,PF,8,8,28.9,5.1,13.4,0.383,1.9,6.8,0.278,3.3,6.6,0.491,0.453,2.8,3.4,0.815,0.9,4.8,3.1,14.9,0.0,0.0,3.0,96.0,1.0,11.0,8.0,3.0,13.0,95.0,2.0,2.0,18.7,0.495,0.075,0.140,0.140,0.140,0.505,0.491,0.750,0.400,0.533,0.400,0.278,0.231,0.533,0.019,1.0,Bigs,0.215,0
99,Naz Reid,25,MIN,C,15,0,24.5,5.1,10.1,0.507,2.1,5.1,0.416,3.0,5.0,0.600,0.612,1.9,2.1,0.906,0.9,1.6,1.1,14.3,0.0,0.0,0.0,46.0,54.0,9.0,6.0,1.0,13.0,62.0,5.0,10.0,15.2,0.493,0.217,0.243,0.033,0.000,0.507,0.600,0.667,0.541,0.600,0.000,0.416,0.667,0.969,0.026,4.0,Bigs,0.460,0
101,Tobias Harris,32,DET,PF,17,17,33.2,5.7,12.5,0.455,1.5,4.4,0.333,4.2,8.1,0.522,0.514,1.4,1.7,0.793,1.1,2.5,1.4,14.2,0.0,0.0,1.0,97.0,3.0,5.0,10.0,7.0,16.0,106.0,6.0,14.0,13.7,0.648,0.174,0.263,0.155,0.056,0.352,0.522,0.784,0.446,0.364,0.500,0.333,0.514,0.920,0.047,10.0,Bigs,0.437,0


## Final Insights
* 